# Setup

In [ ]:
# Import libraries
import os
import warnings
warnings.filterwarnings("ignore")

# Set up environment variables
#project_id='arctic-welder-307905' # change to your id
project_id='tesimimic' # change to your id
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as ss
import numpy as np
np.random.seed(42)

import pandas as pd
pd.set_option('mode.chained_assignment', None)

# authenticate
auth.authenticate_user()

# Data Extraction

In [ ]:
client = bigquery.Client(project=project_id)

# MIMIC 4 Treatment cohort

First icustay of the patients in which he received just 1 blood sac

In [ ]:
#patients trated with a blood sac in mimic 4
tx_mimic4 = client.query("""
WITH all_criteria AS (SELECT DISTINCT(stay_id)
FROM `physionet-data.mimic_derived.sofa`
WHERE stay_id IN(SELECT DISTINCT(stay_id)
                    FROM `physionet-data.mimic_derived.icustay_detail`
                    WHERE icustay_seq = 1
                    AND first_icu_stay= TRUE
                    AND (admission_age BETWEEN 18 AND 89)
  AND stay_id NOT IN(SELECT DISTINCT(i.stay_id)
        FROM `physionet-data.mimic_hosp.diagnoses_icd` d
        INNER JOIN `physionet-data.mimic_icu.icustays` i
            ON d.hadm_id=i.hadm_id
        INNER JOIN `physionet-data.mimic_derived.icustay_detail` a
            ON d.HADM_ID=a.hadm_id
        WHERE icd_code IN (
                --ref: https://www.bmj.com/content/bmj/suppl/2015/02/03/bmj.h246.DC1/junm020747.ww1_default.pdf
                --intracranial bleeding
                '430', --Subarachnoid
                '431', --intracerebral
                '4320', --other and unspecified intracranial bleeding
                '4321', --subdural
                '4329', --unspecified intracranial bleeding
                --upper gastrointestinal
                '5310', --acute gastric ulcer with bleeding without obstruction
                '5312', --acute gastric ulcer with bleeding and perforation without obstruction
                '5314', --chronic or unspecified gastric ulcer with bleeding without obstruction
                '5316', --chronic or unspecified gastric ulcer with bleeding and perforation without obstruction
                '5320', --acute duodenal ulcer with bleeding without obstruction
                '5322', --acute duodenal ulcer with bleeding and perforation without obstruction
                '5324', --chronic or unspecified duodenal ulcer with bleeding without obstruction
                '5326', --chronic or unspecified duodenal ulcer with bleeding and perforation without obstruction
                '5330', --acute peptic ulcer of unspecified site with bleeding without obstruction
                '5332', --acute peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5334', --chronic or unspecified peptic ulcer of unspecified site with bleeding without obstruction
                '5336', --chronic or unspecified peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5340', --acute gastrojejunal ulcer with bleeding without obstruction
                '5342', --acute gastrojejunal ulcer with bleeding and perforation without obstruction
                '5344', --chronic or unspecified gastrojejunal ulcer with bleeding without obstruction
                '5346', --chronic or unspecified gastrojejunal ulcer with bleeding and perforation without obstruction
                '5780', --hematemesis
                '5781', --blood in stool
                '5789', --bleeding of gastrointestinal tract unspecified
                --Lower GI
                '5693', --bleeding of rectum and anus
                --Other Bleeding
                '2878', --other unspecified hemorrhagic conditions
                '2879', --unspecified hemorrhagic conditions
                '5967', --bleeding into bladder wall
                '7848', --bleeding from throat
                '5997', --hematuria, unspecified
                '6271', --postmenopausal bleeding
                '4590', --bleeding unspecified
                '7191', --hemarthrosis site unspecified
                '7863', --hemoptysis, unspecified
                '72992', --nontraumatic hematoma soft tissue
                '9582' --secondary or recurrent following trauma
            )
        OR icd_code LIKE '900%' --vessel injuries
        OR icd_code LIKE '901%' --vessel injuries
        OR icd_code LIKE '902%' --vessel injuries
        OR icd_code LIKE '903%' --vessel injuries
        OR icd_code LIKE '904%' --vessel injuries

        OR UPPER(LAST_CAREUNIT) LIKE '%TSICU%'
        OR UPPER(FIRST_CAREUNIT) LIKE '%TSICU%' )) )
, demographics AS (
    SELECT
        ie.subject_id,
        ie.hadm_id,
        ie.stay_id -- patient level factors
,
        pat.gender,
        pat.dod -- hospital level factors
,
        adm.admittime,
        adm.dischtime,
        pat.anchor_age AS age,
        adm.admission_type,
        adm.hospital_expire_flag,
        adm.deathtime,
        DENSE_RANK() OVER (
            PARTITION BY adm.subject_id
            ORDER BY
                adm.admittime
        ) AS hospstay_seq,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY adm.subject_id
                ORDER BY
                    adm.admittime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_hosp_stay -- icu level factors
,
        ie.intime,
        ie.outtime,
        DENSE_RANK() OVER (
            PARTITION BY ie.hadm_id
            ORDER BY
                ie.intime
        ) AS icustay_seq -- first ICU stay *for the current hospitalization*
,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id
                ORDER BY
                    ie.intime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_icu_stay
    FROM
        `physionet-data.mimic_icu.icustays` ie
        INNER JOIN `physionet-data.mimic_core.admissions` adm ON ie.hadm_id = adm.hadm_id
        INNER JOIN `physionet-data.mimic_core.patients` pat ON ie.subject_id = pat.subject_id
    ORDER BY
        ie.subject_id,
        adm.admittime,
        ie.intime
)
,
first_adm_offset AS (
    SELECT
        stay_id AS id,
        intime,
        outtime
    FROM
        demographics
    WHERE
        first_icu_stay = TRUE),
pivoted_sofa AS (
    SELECT
        *
    FROM
        `physionet-data.mimic_derived.sofa` ps
        LEFT JOIN first_adm_offset f ON ps.stay_id = f.id
        AND ps.endtime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
pivoted_lab AS (
    SELECT
        *
    FROM
        (
            WITH pivoted_lab AS (
                -- create a table which has fuzzy boundaries on ICU admission (+- 12 hours from documented time)
                -- this is used to assign icustay_id to lab data, which can be collected outside ICU
                -- involves first creating a lag/lead version of intime/outtime
                WITH i AS (
                    SELECT
                        subject_id,
                        stay_id,
                        intime,
                        outtime,
                        lag (outtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS outtime_lag,
                        lead (intime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS intime_lead
                    FROM
                        `physionet-data.mimic_icu.icustays`
                ),
                iid_assign AS (
                    SELECT
                        i.subject_id,
                        i.stay_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN i.outtime_lag IS NOT NULL
                            AND i.outtime_lag > DATETIME_SUB(i.intime, INTERVAL 24 HOUR) THEN DATETIME_SUB(
                                i.intime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime, i.outtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(i.intime, INTERVAL 12 HOUR)
                        END AS data_start,
                        CASE
                            WHEN i.intime_lead IS NOT NULL
                            AND i.intime_lead < DATETIME_ADD(i.outtime, INTERVAL 24 HOUR) THEN DATETIME_ADD(
                                i.outtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime_lead, i.outtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(i.outtime, INTERVAL 12 HOUR)
                        END AS data_end
                    FROM
                        i
                ) -- also create fuzzy boundaries on hospitalization
,
                h AS (
                    SELECT
                        subject_id,hadm_id,admittime,dischtime,
                        lag (dischtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS dischtime_lag,
                        lead (admittime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS admittime_lead
                    FROM
                        `physionet-data.mimic_core.admissions`
                ),
                adm AS (
                    SELECT
                        h.subject_id,
                        h.hadm_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN h.dischtime_lag IS NOT NULL
                            AND h.dischtime_lag > DATETIME_SUB(h.admittime, INTERVAL '24' HOUR) THEN DATETIME_SUB(
                                h.admittime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime, h.dischtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(h.admittime, INTERVAL '12' HOUR)
                        END AS data_start,
                        CASE
                            WHEN h.admittime_lead IS NOT NULL
                            AND h.admittime_lead < DATETIME_ADD(h.dischtime, INTERVAL '24' HOUR) THEN DATETIME_ADD(
                                h.dischtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime_lead, h.dischtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(h.dischtime, INTERVAL '12' HOUR)
                        END AS data_end
                    FROM
                        h
                ),
                le AS (
                    -- begin query that extracts the data
                    SELECT
                        subject_id,
                        charttime -- here we assign labels to ITEMIDs
                        -- this also fuses together multiple ITEMIDs containing the same data
,
                        CASE
                            WHEN itemid = 50868 THEN 'ANION GAP'
                            WHEN itemid = 50862 THEN 'ALBUMIN'
                            WHEN itemid = 51144 THEN 'BANDS'
                            WHEN itemid = 50882 THEN 'BICARBONATE'
                            WHEN itemid = 50885 THEN 'BILIRUBIN'
                            WHEN itemid = 50912 THEN 'CREATININE' -- exclude blood gas
                            -- WHEN itemid = 50806 THEN 'CHLORIDE'
                            WHEN itemid = 50902 THEN 'CHLORIDE' -- exclude blood gas
                            -- WHEN itemid = 50809 THEN 'GLUCOSE'
                            WHEN itemid = 50931 THEN 'GLUCOSE' -- exclude blood gas
                            --WHEN itemid = 50810 THEN 'HEMATOCRIT'
                            WHEN itemid = 51221 THEN 'HEMATOCRIT' -- exclude blood gas
                            --WHEN itemid = 50811 THEN 'HEMOGLOBIN'
                            WHEN itemid = 51222 THEN 'HEMOGLOBIN'
                            WHEN itemid = 50813 THEN 'LACTATE'
                            WHEN itemid = 51265 THEN 'PLATELET' -- exclude blood gas
                            -- WHEN itemid = 50822 THEN 'POTASSIUM'
                            WHEN itemid = 50971 THEN 'POTASSIUM'
                            WHEN itemid = 51275 THEN 'PTT'
                            WHEN itemid = 51237 THEN 'INR'
                            WHEN itemid = 51274 THEN 'PT' -- exclude blood gas
                            -- WHEN itemid = 50824 THEN 'SODIUM'
                            WHEN itemid = 50983 THEN 'SODIUM'
                            WHEN itemid = 51006 THEN 'BUN'
                            WHEN itemid = 51300 THEN 'WBC'
                            WHEN itemid = 51301 THEN 'WBC'
                            ELSE NULL
                        END AS label,
                        -- add in some sanity checks on the values
                        -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
                        CASE
                            WHEN itemid = 50862
                            AND valuenum > 10 THEN NULL -- g/dL 'ALBUMIN'
                            WHEN itemid = 50868
                            AND valuenum > 10000 THEN NULL -- mEq/L 'ANION GAP'
                            WHEN itemid = 51144
                            AND valuenum < 0 THEN NULL -- immature band forms, %
                            WHEN itemid = 51144
                            AND valuenum > 100 THEN NULL -- immature band forms, %
                            WHEN itemid = 50882
                            AND valuenum > 10000 THEN NULL -- mEq/L 'BICARBONATE'
                            WHEN itemid = 50885
                            AND valuenum > 150 THEN NULL -- mg/dL 'BILIRUBIN'
                            WHEN itemid = 50806
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50902
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50912
                            AND valuenum > 150 THEN NULL -- mg/dL 'CREATININE'
                            WHEN itemid = 50809
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50931
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50810
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 51221
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 50811
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 51222
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 50813
                            AND valuenum > 50 THEN NULL -- mmol/L 'LACTATE'
                            WHEN itemid = 51265
                            AND valuenum > 10000 THEN NULL -- K/uL 'PLATELET'
                            WHEN itemid = 50822
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 50971
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 51275
                            AND valuenum > 150 THEN NULL -- sec 'PTT'
                            WHEN itemid = 51237
                            AND valuenum > 50 THEN NULL -- 'INR'
                            WHEN itemid = 51274
                            AND valuenum > 150 THEN NULL -- sec 'PT'
                            WHEN itemid = 50824
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 50983
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 51006
                            AND valuenum > 300 THEN NULL -- 'BUN'
                            WHEN itemid = 51300
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            WHEN itemid = 51301
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            ELSE valuenum
                        END AS valuenum
                    FROM
                        `physionet-data.mimic_hosp.labevents`
                    WHERE
                        ITEMID IN (
                            -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
                            50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
                            50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
                            51144, -- BANDS - hematology
                            50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
                            50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
                            50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
                            50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
                            -- 50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
                            50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
                            -- 50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
                            51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
                            -- 50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
                            51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
                            -- 50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
                            50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
                            51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
                            50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
                            -- 50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
                            51275, -- PTT | HEMATOLOGY | BLOOD | 474937
                            51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
                            51274, -- PT | HEMATOLOGY | BLOOD | 469090
                            50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
                            -- 50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
                            51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
                            51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
                            51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
                        )
                        AND valuenum IS NOT NULL
                        AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
                ),
                le_avg AS (
                    SELECT
                        le.subject_id,
                        le.charttime,
                        avg(
                            CASE
                                WHEN label = 'ANION GAP' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ANIONGAP,
                        avg(
                            CASE
                                WHEN label = 'ALBUMIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ALBUMIN,
                        avg(
                            CASE
                                WHEN label = 'BANDS' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BANDS,
                        avg(
                            CASE
                                WHEN label = 'BICARBONATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BICARBONATE,
                        avg(
                            CASE
                                WHEN label = 'BILIRUBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BILIRUBIN,
                        avg(
                            CASE
                                WHEN label = 'CREATININE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CREATININE,
                        avg(
                            CASE
                                WHEN label = 'CHLORIDE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CHLORIDE,
                        avg(
                            CASE
                                WHEN label = 'GLUCOSE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS GLUCOSE,
                        avg(
                            CASE
                                WHEN label = 'HEMATOCRIT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMATOCRIT,
                        avg(
                            CASE
                                WHEN label = 'HEMOGLOBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMOGLOBIN,
                        avg(
                            CASE
                                WHEN label = 'LACTATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS LACTATE,
                        avg(
                            CASE
                                WHEN label = 'PLATELET' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PLATELET,
                        avg(
                            CASE
                                WHEN label = 'POTASSIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS POTASSIUM,
                        avg(
                            CASE
                                WHEN label = 'PTT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PTT,
                        avg(
                            CASE
                                WHEN label = 'INR' THEN valuenum
                                ELSE NULL
                            END
                        ) AS INR,
                        avg(
                            CASE
                                WHEN label = 'PT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PT,
                        avg(
                            CASE
                                WHEN label = 'SODIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS SODIUM,
                        avg(
                            CASE
                                WHEN label = 'BUN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BUN,
                        avg(
                            CASE
                                WHEN label = 'WBC' THEN valuenum
                                ELSE NULL
                            END
                        ) AS WBC
                    FROM
                        le
                    GROUP BY
                        le.subject_id,
                        le.charttime
                )
                SELECT
                    iid.stay_id,
                    adm.hadm_id,
                    le_avg.*
                FROM
                    le_avg
                    LEFT JOIN adm ON le_avg.subject_id = adm.subject_id
                    AND le_avg.charttime >= adm.data_start
                    AND le_avg.charttime < adm.data_end
                    LEFT JOIN iid_assign iid ON le_avg.subject_id = iid.subject_id
                    AND le_avg.charttime >= iid.data_start
                    AND le_avg.charttime < iid.data_end
                ORDER BY
                    le_avg.subject_id,
                    le_avg.charttime
            )
            SELECT
                *
            FROM
                pivoted_lab
        ) AS pl
        LEFT JOIN first_adm_offset f ON pl.stay_id = f.id
        AND pl.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
)
,
mechvent AS (
    SELECT
        stay_id,
        charttime -- case statement determining whether it is an instance of mech vent
,
        MAX(
            CASE
                WHEN itemid IS NULL
                OR value IS NULL THEN 0 -- can't have null values
                WHEN itemid = 720
                AND value != 'Other/Remarks' THEN 1 -- VentTypeRecorded
                WHEN itemid = 223848
                AND value != 'Other' THEN 1
                WHEN itemid = 223849 THEN 1 -- ventilator mode
                WHEN itemid = 467
                AND value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
                WHEN itemid IN(
                    445,
                    448,
                    449,
                    450,
                    1340,
                    1486,
                    1600,
                    224687 -- minute volume
,
                    639,
                    654,
                    681,
                    682,
                    683,
                    684,
                    224685,
                    224684,
                    224686 -- tidal volume
,
                    218,
                    436,
                    535,
                    444,
                    459,
                    224697,
                    224695,
                    224696,
                    224746,
                    224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
,
                    221,
                    1,
                    1211,
                    1655,
                    2000,
                    226873,
                    224738,
                    224419,
                    224750,
                    227187 -- Insp pressure
,
                    543 -- PlateauPressure
,
                    5865,
                    5866,
                    224707,
                    224709,
                    224705,
                    224706 -- APRV pressure
,
                    60,
                    437,
                    505,
                    506,
                    686,
                    220339,
                    224700 -- PEEP
,
                    3459 -- high pressure relief
,
                    501,
                    502,
                    503,
                    224702 -- PCV
,
                    223,
                    667,
                    668,
                    669,
                    670,
                    671,
                    672 -- TCPCV
,
                    224701 -- PSVlevel
                ) THEN 1
                ELSE 0
            END
        ) AS ventmarker
    FROM
        `physionet-data.mimic_icu.chartevents` ce
        LEFT JOIN first_adm_offset f ON ce.stay_id = f.id
        AND ce.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    WHERE
        ce.value IS NOT NULL -- exclude rows marked as error
        --AND ce.error != 1 -- the below are settings used to indicate ventilation
        AND itemid IN(
            720,
            223849 -- vent mode
,
            223848 -- vent type
,
            445,
            448,
            449,
            450,
            1340,
            1486,
            1600,
            224687 -- minute volume
,
            639,
            654,
            681,
            682,
            683,
            684,
            224685,
            224684,
            224686 -- tidal volume
,
            218,
            436,
            535,
            444,
            224697,
            224695,
            224696,
            224746,
            224747 -- High/Low/Peak/Mean ("RespPressure")
,
            221,
            1,
            1211,
            1655,
            2000,
            226873,
            224738,
            224419,
            224750,
            227187 -- Insp pressure
,
            543 -- PlateauPressure
,
            5865,
            5866,
            224707,
            224709,
            224705,
            224706 -- APRV pressure
,
            60,
            437,
            505,
            506,
            686,
            220339,
            224700 -- PEEP
,
            3459 -- high pressure relief
,
            501,
            502,
            503,
            224702 -- PCV
,
            223,
            667,
            668,
            669,
            670,
            671,
            672 -- TCPCV
,
            224701 -- PSVlevel
            -- the below are settings used to indicate extubation
,
            640 -- extubated
            -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
,
            468 -- O2 Delivery Device#2
,
            469 -- O2 Delivery Mode
,
            470 -- O2 Flow (lpm)
,
            471 -- O2 Flow (lpm) #2
,
            227287 -- O2 Flow (additional cannula)
,
            226732 -- O2 Delivery Device(s)
,
            223834 -- O2 Flow
            -- used in both oxygen + vent calculation
,
            467 -- O2 Delivery Device
        )
    GROUP BY
        stay_id,
        charttime
)
,
sepsis AS (
    -- ICD-9 codes for Angus criteria of sepsis
    SELECT
        hadm_id,
        MAX(
            CASE
                WHEN SUBSTR(icd_code, 1, 3) IN (
                    '001','002','003','004','005',
                    '008','009','010','011','012',
                    '013','014','015','016','017',
                    '018','020','021','022','023',
                    '024','025','026','027','030',
                    '031','032','033','034','035',
                    '036','037','038','039','040',
                    '041','090','091','092','093',
                    '094','095','096','097','098',
                    '100','101','102','103','104',
                    '110','111','112','114','115',
                    '116','117','118','320','322',
                    '324','325','420','421','451',
                    '461','462','463','464','465',
                    '481','482','485','486','494',
                    '510','513','540','541','542',
                    '566','567','590','597','601',
                    '614','615','616','681','682',
                    '683','686','730'
                ) THEN 1
                WHEN SUBSTR(icd_code, 1, 4) IN (
                    '5695','5720','5721','5750',
                    '5990','7110','7907','9966',
                    '9985','9993'
                ) THEN 1
                WHEN SUBSTR(icd_code, 1, 5) IN (
                    '49121','56201','56203',
                    '56211','56213','56983'
                ) THEN 1
                ELSE 0
            END
        ) AS sepsisflag
    FROM
        `physionet-data.mimic_hosp.diagnoses_icd`
    GROUP BY
        hadm_id
),
--,
--vasopressor_duration AS (
    --SELECT
        --*
    --FROM
        --`physionet-data.mimiciii_derived.vasopressordurations`
--),
premorbid AS (
    WITH icd AS (
        SELECT
            hadm_id,
            seq_num,
            CAST(icd_code AS STRING) AS icd_code
        FROM
            `physionet-data.mimic_hosp.diagnoses_icd`
    ),
    eliflg AS (
        SELECT
            hadm_id,
            seq_num,
            icd_code -- note that these codes will seem incomplete at first
            -- for example, CHF is missing a lot of codes referenced in the literature (402.11, 402.91, etc)
            -- these codes are captured by hypertension flags instead
            -- later there are some complicated rules which confirm/reject those codes as CHF
,
            CASE
                WHEN SUBSTR(icd_code, 1, 3) IN(
                    '410', --Acute myocardial infarction
                    '411', --Other acute and subacute forms of ischemic heart disease
                    '412', --Old myocardial infarction
                    '413', --Angina pectoris
                    '414'  --Other forms of chronic ischemic heart disease
                ) THEN 1
            END AS IHD,
            CASE
                WHEN icd_code = '39891' THEN 1
                WHEN icd_code BETWEEN '4280 '
                AND '4289 ' THEN 1
            END AS CHF
            /* Congestive heart failure */
            -- cardiac arrhythmias is removed in up to date versions
,
            CASE
                WHEN icd_code BETWEEN '4400 '
                AND '4409 ' THEN 1
                WHEN icd_code BETWEEN '44100'
                AND '4419 ' THEN 1
                WHEN icd_code BETWEEN '4420 '
                AND '4429 ' THEN 1
                WHEN icd_code BETWEEN '4431 '
                AND '4439 ' THEN 1
                WHEN icd_code BETWEEN '44421'
                AND '44422' THEN 1
                WHEN icd_code = '4471 ' THEN 1
                WHEN icd_code = '449  ' THEN 1
                WHEN icd_code = '5571 ' THEN 1
                WHEN icd_code = '5579 ' THEN 1
                WHEN icd_code = 'V434 ' THEN 1
            END AS PERIVASC
            /* Peripheral vascular disorder */
,
            CASE
                WHEN icd_code = '40200' THEN 1
                WHEN icd_code = '40210' THEN 1
                WHEN icd_code = '40290' THEN 1
                WHEN icd_code = '40509' THEN 1
                WHEN icd_code = '40519' THEN 1
                WHEN icd_code = '40599' THEN 1
            END AS HTNWOCHF
            /* Hypertensive heart disease without heart failure */
,
            CASE
                WHEN icd_code = '40201' THEN 1
                WHEN icd_code = '40211' THEN 1
                WHEN icd_code = '40291' THEN 1
            END AS HTNWCHF
            /* Hypertensive heart disease with heart failure */
,
            CASE
                WHEN icd_code = '40301' THEN 1
                WHEN icd_code = '40311' THEN 1
                WHEN icd_code = '40391' THEN 1
            END AS HRENWRF
            /* Hypertensive renal disease with renal failure */
,
            CASE
                WHEN icd_code = '40401' THEN 1
                WHEN icd_code = '40411' THEN 1
                WHEN icd_code = '40491' THEN 1
            END AS HHRWCHF
            /* Hypertensive heart and renal disease with heart failure */
,
            CASE
                WHEN icd_code = '40402' THEN 1
                WHEN icd_code = '40412' THEN 1
                WHEN icd_code = '40492' THEN 1
            END AS HHRWRF
            /* Hypertensive heart and renal disease with renal failure */
,
            CASE
                WHEN icd_code = '40403' THEN 1
                WHEN icd_code = '40413' THEN 1
                WHEN icd_code = '40493' THEN 1
            END AS HHRWHRF
            /* Hypertensive heart and renal disease with heart and renal failure */
,
            CASE
                WHEN icd_code = '585  ' THEN 1 -- discontinued code
                WHEN icd_code = '5853 ' THEN 1
                WHEN icd_code = '5854 ' THEN 1
                WHEN icd_code = '5855 ' THEN 1
                WHEN icd_code = '5856 ' THEN 1
                WHEN icd_code = '5859 ' THEN 1
                WHEN icd_code = '586  ' THEN 1
                WHEN icd_code = 'V420 ' THEN 1
                WHEN icd_code = 'V451 ' THEN 1
                WHEN icd_code BETWEEN 'V560 '
                AND 'V5632' THEN 1
                WHEN icd_code = 'V568 ' THEN 1
                WHEN icd_code BETWEEN 'V4511'
                AND 'V4512' THEN 1
            END AS RENLFAIL
            /* Renal failure */
        FROM
            icd
    ),
    eligrp AS (
        SELECT
            hadm_id,
            MAX(ihd) AS ihd,
            MAX(chf) AS chf,
            MAX(perivasc) AS perivasc,
            MAX(htnwchf) AS htnwchf,
            MAX(hrenwrf) AS hrenwrf,
            MAX(hhrwchf) AS hhrwchf,
            MAX(hhrwrf) AS hhrwrf,
            MAX(hhrwhrf) AS hhrwhrf,
            MAX(renlfail) AS renlfail
        FROM
            eliflg
        GROUP BY
            hadm_id
    )
    SELECT
        adm.subject_id,
        adm.hadm_id,
        CASE
            WHEN ihd = 1 THEN 1
            ELSE 0
        END AS ISCHEMIC_HEART_DISEASE,
        CASE
            WHEN chf = 1 THEN 1
            WHEN htnwchf = 1 THEN 1
            WHEN hhrwchf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS CONGESTIVE_HEART_FAILURE,
        CASE
            WHEN perivasc = 1 THEN 1
            ELSE 0
        END AS PERIPHERAL_VASCULAR,
        CASE
            WHEN renlfail = 1 THEN 1
            WHEN hrenwrf = 1 THEN 1
            WHEN hhrwrf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS RENAL_FAILURE
    FROM
        `physionet-data.mimic_core.admissions` adm
        LEFT JOIN eligrp eli ON adm.hadm_id = eli.hadm_id
    ORDER BY
        adm.hadm_id
),
surgflag AS (
    SELECT
        adm.hadm_id,
        CASE
            WHEN LOWER(curr_service) LIKE '%surg%' THEN 1
            ELSE 0
        END AS surgical,
        ROW_NUMBER() OVER(
            PARTITION BY adm.HADM_ID
            ORDER BY
                TRANSFERTIME
        ) AS serviceOrder
    FROM
        `physionet-data.mimic_core.admissions` adm
        LEFT JOIN `physionet-data.mimic_hosp.services` se ON adm.hadm_id = se.hadm_id
)
,
tx_raw AS (
    -- BLOOD TRANSFUSED in mL
    -- carevue not in MIMIC 4
    -- include all MIMIC 4 add carvue from MIMIC 3
    WITH inputevents AS (
        SELECT
            ipe.stay_id,
            ipe.starttime AS charttime,
            ROUND(ipe.amount) AS amount,
        FROM
            `physionet-data.mimic_icu.inputevents` ipe
        WHERE
            ipe.itemid IN(
                225168, -- Packed Red Blood Cells
                220996, -- Packed Red Cells
                221013,  -- Whole blood
                226368, -- OR Packed RBC Intake
                227070 -- PACU Packed RBC Intake
            )
            AND amount > 100
            --AND ipe.statusdescription != 'Rewritten'
    )
    SELECT
        ipe.stay_id,
        charttime,
        SUM(amount) AS rbc_volume,
    FROM
        inputevents ipe
    GROUP BY
        ipe.stay_id,
        ipe.charttime
),
tx AS (
    SELECT
        stay_id,
        TIMESTAMP_SECONDS(60 * DIV(UNIX_SECONDS(TIMESTAMP(charttime)), 60)) AS episode,
        --some samples have duplicate readings every 15 minutes - we take an hourly window and average the volume across this window
        --doing so reduces errors significantly
        MIN(charttime) AS charttime,
        --get the first timestamp of each window
        AVG(rbc_volume) AS rbc_volume
    FROM tx_raw
    GROUP BY stay_id, episode
    ORDER BY stay_id, episode
),
tx_counts AS (
    SELECT
        stay_id,
        COUNT(charttime) AS tx_episodes,
        SUM(rbc_volume) AS rbc_volume,
        CEIL(SUM(rbc_volume) / 350) AS rbc_units
    FROM tx
    WHERE
        stay_id IS NOT NULL
    GROUP BY
        stay_id
),
pivoted_hemoglobin AS (
    SELECT
        stay_id,
        charttime,
        hemoglobin
    FROM
        pivoted_lab
    WHERE
        stay_id IS NOT NULL
        AND charttime IS NOT NULL
        AND hemoglobin IS NOT NULL
    ORDER BY
        stay_id
)
, tx_hgb_paired AS (
        SELECT
            w.stay_id,
            w.last_hb_in_window,
            MIN(p.hemoglobin) AS hemoglobin,
            w.tx_offset,
            w.rbc_volume,
            DATETIME_DIFF(w.tx_offset, w.last_hb_in_window, SECOND) AS delta_tx_hgb_seconds,
        FROM
            (
                SELECT
                    p.stay_id,
                    t.charttime AS tx_offset,
                    t.rbc_volume,
                    MAX(p.charttime) AS last_hb_in_window,
                    p.hemoglobin
                FROM tx t --`arctic-welder-307905.blood_transfusion.tx` t
                    INNER JOIN pivoted_hemoglobin p ON p.stay_id = t.stay_id
                    AND p.charttime BETWEEN DATETIME_SUB(t.charttime, INTERVAL 6 HOUR)
                    AND t.charttime --we search the 6 hours prior to transfusion for minimum hemoglobin levels associated with that transfusion
                GROUP BY
                    p.stay_id,
                    t.charttime,
                    t.rbc_volume,
                    p.hemoglobin
                ORDER BY
                    stay_id,
                    last_hb_in_window
            ) w
            INNER JOIN pivoted_hemoglobin p ON p.stay_id = w.stay_id
            AND p.charttime = w.last_hb_in_window
        GROUP BY
            w.stay_id,
            w.last_hb_in_window,
            w.tx_offset,
            w.rbc_volume
        ORDER BY
            stay_id,
             last_hb_in_window,
             tx_offset
)
, tx_hgb_filtered AS (
        SELECT
            *,
            ROW_NUMBER() OVER(x) AS event_number
        FROM tx_hgb_paired
        WINDOW x AS(
                PARTITION BY stay_id
                ORDER BY tx_offset
            )
        ORDER BY stay_id

)
, closest_tx_hb_pairing AS (SELECT stay_id,
      ROW_NUMBER() OVER(PARTITION BY stay_id ORDER BY tx_offset) AS tx_episode,
     last_hb_in_window, hemoglobin,
      tx_offset, rbc_volume,
      delta_tx_hgb_seconds/60 AS delta_offset_minutes
    FROM tx_hgb_filtered
    ORDER BY stay_id, event_number),

closest_tx_hb_pairing_icu_time AS (SELECT a.*,b.INTIME,b.OUTTIME,ROUND((rbc_volume/ 350),0) AS rbc_units
FROM closest_tx_hb_pairing  a
INNER JOIN  `physionet-data.mimic_icu.icustays` b
ON a.stay_id= b.stay_id
AND a.tx_offset BETWEEN b.INTIME and b.OUTTIME
ORDER BY a.stay_id)

SELECT * FROM closest_tx_hb_pairing_icu_time
""").to_dataframe()

In [ ]:
df1= tx_mimic4.copy()
df1.sort_values(by = ['stay_id', 'tx_offset','delta_offset_minutes'], ascending = [True, True,True], inplace=True)
#Keep only the first ICU stay out of all the possible stays
df1.drop_duplicates(subset=['stay_id'], keep= 'first', inplace=True)
#Consider only patient that received just 1 blood unit
treatment_mimic4_1rbc = df1.loc[df1['rbc_units'] == 1.0]
#First ICUstay for the patients treated with just 1 blood unit, mimic 4
treatment_mimic4_1rbc.head()

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units
0,30001446,1,2186-04-12 02:00:00,7.9,2186-04-12 04:28:00,375.0,148.0,2186-04-12 03:49:00,2186-04-13 19:45:47,1.0
1,30001555,1,2177-09-27 11:50:00,7.6,2177-09-27 16:00:00,350.0,250.0,2177-09-27 11:23:13,2177-09-28 18:26:00,1.0
3,30001947,1,2162-12-26 18:25:00,6.9,2162-12-26 21:04:00,375.0,159.0,2162-12-26 15:04:30,2162-12-27 22:31:00,1.0
7,30003372,1,2170-06-11 04:17:00,7.8,2170-06-11 08:45:00,350.0,268.0,2170-06-10 03:15:10,2170-06-12 15:54:41,1.0
9,30003749,2,2120-11-05 16:30:00,6.3,2120-11-05 17:48:00,375.0,78.0,2120-11-05 16:03:00,2120-11-07 08:19:44,1.0


# Carevue MIMIC 3 Treatment
First icustay of the patients in which he received just 1 blood sac

In [ ]:
#patients trated with a blood sac in mimic 3 carevue (metavision has overlaps with mimic 4)
tx_mimic3_carevue = client.query("""
WITH
all_criteria AS (SELECT DISTINCT(icustay_id)
FROM `physionet-data.mimiciii_derived.pivoted_sofa`
WHERE icustay_id IN(SELECT DISTINCT(icustay_id)
                    FROM `physionet-data.mimiciii_derived.icustay_detail`
                    WHERE icustay_seq = 1
                    AND first_icu_stay= TRUE
                    AND (admission_age BETWEEN 18 AND 89)
                    )
    AND icustay_id NOT IN(SELECT DISTINCT(i.ICUSTAY_ID)
        FROM `physionet-data.mimiciii_clinical.diagnoses_icd` d
        INNER JOIN `physionet-data.mimiciii_clinical.icustays` i
            ON d.hadm_id=i.hadm_id
        INNER JOIN `physionet-data.mimiciii_derived.icustay_detail` a
            ON d.HADM_ID=a.hadm_id
        WHERE icd9_code IN (
                --ref: https://www.bmj.com/content/bmj/suppl/2015/02/03/bmj.h246.DC1/junm020747.ww1_default.pdf
                --intracranial bleeding
                '430', --Subarachnoid
                '431', --intracerebral
                '4320', --other and unspecified intracranial bleeding
                '4321', --subdural
                '4329', --unspecified intracranial bleeding
                --upper gastrointestinal
                '5310', --acute gastric ulcer with bleeding without obstruction
                '5312', --acute gastric ulcer with bleeding and perforation without obstruction
                '5314', --chronic or unspecified gastric ulcer with bleeding without obstruction
                '5316', --chronic or unspecified gastric ulcer with bleeding and perforation without obstruction
                '5320', --acute duodenal ulcer with bleeding without obstruction
                '5322', --acute duodenal ulcer with bleeding and perforation without obstruction
                '5324', --chronic or unspecified duodenal ulcer with bleeding without obstruction
                '5326', --chronic or unspecified duodenal ulcer with bleeding and perforation without obstruction
                '5330', --acute peptic ulcer of unspecified site with bleeding without obstruction
                '5332', --acute peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5334', --chronic or unspecified peptic ulcer of unspecified site with bleeding without obstruction
                '5336', --chronic or unspecified peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5340', --acute gastrojejunal ulcer with bleeding without obstruction
                '5342', --acute gastrojejunal ulcer with bleeding and perforation without obstruction
                '5344', --chronic or unspecified gastrojejunal ulcer with bleeding without obstruction
                '5346', --chronic or unspecified gastrojejunal ulcer with bleeding and perforation without obstruction
                '5780', --hematemesis
                '5781', --blood in stool
                '5789', --bleeding of gastrointestinal tract unspecified
                --Lower GI
                '5693', --bleeding of rectum and anus
                --Other Bleeding
                '2878', --other unspecified hemorrhagic conditions
                '2879', --unspecified hemorrhagic conditions
                '5967', --bleeding into bladder wall
                '7848', --bleeding from throat
                '5997', --hematuria, unspecified
                '6271', --postmenopausal bleeding
                '4590', --bleeding unspecified
                '7191', --hemarthrosis site unspecified
                '7863', --hemoptysis, unspecified
                '72992', --nontraumatic hematoma soft tissue
                '9582' --secondary or recurrent following trauma
            )
        OR icd9_code LIKE '900%' --vessel injuries
        OR icd9_code LIKE '901%' --vessel injuries
        OR icd9_code LIKE '902%' --vessel injuries
        OR icd9_code LIKE '903%' --vessel injuries
        OR icd9_code LIKE '904%' --vessel injuries

        OR UPPER(LAST_CAREUNIT) LIKE '%TSICU%'
        OR UPPER(FIRST_CAREUNIT) LIKE '%TSICU%' ))
 ,
 demographics AS (
    SELECT
        ie.subject_id,
        ie.hadm_id,
        ie.icustay_id -- patient level factors
,
        pat.gender,
        pat.dod -- hospital level factors
,
        adm.admittime,
        adm.dischtime,
        DATETIME_DIFF(adm.admittime, pat.dob, YEAR) AS age,
        adm.admission_type,
        adm.hospital_expire_flag,
        adm.deathtime,
        DENSE_RANK() OVER (
            PARTITION BY adm.subject_id
            ORDER BY
                adm.admittime
        ) AS hospstay_seq,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY adm.subject_id
                ORDER BY
                    adm.admittime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_hosp_stay -- icu level factors
,
        ie.intime,
        ie.outtime,
        DENSE_RANK() OVER (
            PARTITION BY ie.hadm_id
            ORDER BY
                ie.intime
        ) AS icustay_seq -- first ICU stay *for the current hospitalization*
,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id
                ORDER BY
                    ie.intime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_icu_stay
    FROM
        `physionet-data.mimiciii_clinical.icustays` ie
        INNER JOIN `physionet-data.mimiciii_clinical.admissions` adm ON ie.hadm_id = adm.hadm_id
        INNER JOIN `physionet-data.mimiciii_clinical.patients` pat ON ie.subject_id = pat.subject_id
    ORDER BY
        ie.subject_id,
        adm.admittime,
        ie.intime
),
first_adm_offset AS (
    SELECT
        icustay_id AS id,
        intime,
        outtime
    FROM
        demographics
    WHERE
        first_icu_stay = TRUE
),
pivoted_sofa AS (
    SELECT
        *
    FROM
        `physionet-data.mimiciii_derived.pivoted_sofa` ps
        LEFT JOIN first_adm_offset f ON ps.icustay_id = f.id
        AND ps.endtime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
pivoted_lab AS (
    SELECT
        *
    FROM
        (
            WITH pivoted_lab AS (
                -- create a table which has fuzzy boundaries on ICU admission (+- 12 hours from documented time)
                -- this is used to assign icustay_id to lab data, which can be collected outside ICU
                -- involves first creating a lag/lead version of intime/outtime
                WITH i AS (
                    SELECT
                        subject_id,
                        icustay_id,
                        intime,
                        outtime,
                        lag (outtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS outtime_lag,
                        lead (intime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS intime_lead
                    FROM
                        `physionet-data.mimiciii_clinical.icustays`
                ),
                iid_assign AS (
                    SELECT
                        i.subject_id,
                        i.icustay_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN i.outtime_lag IS NOT NULL
                            AND i.outtime_lag > DATETIME_SUB(i.intime, INTERVAL 24 HOUR) THEN DATETIME_SUB(
                                i.intime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime, i.outtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(i.intime, INTERVAL 12 HOUR)
                        END AS data_start,
                        CASE
                            WHEN i.intime_lead IS NOT NULL
                            AND i.intime_lead < DATETIME_ADD(i.outtime, INTERVAL 24 HOUR) THEN DATETIME_ADD(
                                i.outtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime_lead, i.outtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(i.outtime, INTERVAL 12 HOUR)
                        END AS data_end
                    FROM
                        i
                ) -- also create fuzzy boundaries on hospitalization
,
                h AS (
                    SELECT
                        subject_id,hadm_id,admittime,dischtime,
                        lag (dischtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS dischtime_lag,
                        lead (admittime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS admittime_lead
                    FROM
                        `physionet-data.mimiciii_clinical.admissions`
                ),
                adm AS (
                    SELECT
                        h.subject_id,
                        h.hadm_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN h.dischtime_lag IS NOT NULL
                            AND h.dischtime_lag > DATETIME_SUB(h.admittime, INTERVAL '24' HOUR) THEN DATETIME_SUB(
                                h.admittime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime, h.dischtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(h.admittime, INTERVAL '12' HOUR)
                        END AS data_start,
                        CASE
                            WHEN h.admittime_lead IS NOT NULL
                            AND h.admittime_lead < DATETIME_ADD(h.dischtime, INTERVAL '24' HOUR) THEN DATETIME_ADD(
                                h.dischtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime_lead, h.dischtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(h.dischtime, INTERVAL '12' HOUR)
                        END AS data_end
                    FROM
                        h
                ),
                le AS (
                    -- begin query that extracts the data
                    SELECT
                        subject_id,
                        charttime -- here we assign labels to ITEMIDs
                        -- this also fuses together multiple ITEMIDs containing the same data
,
                        CASE
                            WHEN itemid = 50868 THEN 'ANION GAP'
                            WHEN itemid = 50862 THEN 'ALBUMIN'
                            WHEN itemid = 51144 THEN 'BANDS'
                            WHEN itemid = 50882 THEN 'BICARBONATE'
                            WHEN itemid = 50885 THEN 'BILIRUBIN'
                            WHEN itemid = 50912 THEN 'CREATININE' -- exclude blood gas
                            -- WHEN itemid = 50806 THEN 'CHLORIDE'
                            WHEN itemid = 50902 THEN 'CHLORIDE' -- exclude blood gas
                            -- WHEN itemid = 50809 THEN 'GLUCOSE'
                            WHEN itemid = 50931 THEN 'GLUCOSE' -- exclude blood gas
                            --WHEN itemid = 50810 THEN 'HEMATOCRIT'
                            WHEN itemid = 51221 THEN 'HEMATOCRIT' -- exclude blood gas
                            --WHEN itemid = 50811 THEN 'HEMOGLOBIN'
                            WHEN itemid = 51222 THEN 'HEMOGLOBIN'
                            WHEN itemid = 50813 THEN 'LACTATE'
                            WHEN itemid = 51265 THEN 'PLATELET' -- exclude blood gas
                            -- WHEN itemid = 50822 THEN 'POTASSIUM'
                            WHEN itemid = 50971 THEN 'POTASSIUM'
                            WHEN itemid = 51275 THEN 'PTT'
                            WHEN itemid = 51237 THEN 'INR'
                            WHEN itemid = 51274 THEN 'PT' -- exclude blood gas
                            -- WHEN itemid = 50824 THEN 'SODIUM'
                            WHEN itemid = 50983 THEN 'SODIUM'
                            WHEN itemid = 51006 THEN 'BUN'
                            WHEN itemid = 51300 THEN 'WBC'
                            WHEN itemid = 51301 THEN 'WBC'
                            ELSE NULL
                        END AS label,
                        -- add in some sanity checks on the values
                        -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
                        CASE
                            WHEN itemid = 50862
                            AND valuenum > 10 THEN NULL -- g/dL 'ALBUMIN'
                            WHEN itemid = 50868
                            AND valuenum > 10000 THEN NULL -- mEq/L 'ANION GAP'
                            WHEN itemid = 51144
                            AND valuenum < 0 THEN NULL -- immature band forms, %
                            WHEN itemid = 51144
                            AND valuenum > 100 THEN NULL -- immature band forms, %
                            WHEN itemid = 50882
                            AND valuenum > 10000 THEN NULL -- mEq/L 'BICARBONATE'
                            WHEN itemid = 50885
                            AND valuenum > 150 THEN NULL -- mg/dL 'BILIRUBIN'
                            WHEN itemid = 50806
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50902
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50912
                            AND valuenum > 150 THEN NULL -- mg/dL 'CREATININE'
                            WHEN itemid = 50809
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50931
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50810
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 51221
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 50811
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 51222
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 50813
                            AND valuenum > 50 THEN NULL -- mmol/L 'LACTATE'
                            WHEN itemid = 51265
                            AND valuenum > 10000 THEN NULL -- K/uL 'PLATELET'
                            WHEN itemid = 50822
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 50971
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 51275
                            AND valuenum > 150 THEN NULL -- sec 'PTT'
                            WHEN itemid = 51237
                            AND valuenum > 50 THEN NULL -- 'INR'
                            WHEN itemid = 51274
                            AND valuenum > 150 THEN NULL -- sec 'PT'
                            WHEN itemid = 50824
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 50983
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 51006
                            AND valuenum > 300 THEN NULL -- 'BUN'
                            WHEN itemid = 51300
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            WHEN itemid = 51301
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            ELSE valuenum
                        END AS valuenum
                    FROM
                        `physionet-data.mimiciii_clinical.labevents`
                    WHERE
                        ITEMID IN (
                            -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
                            50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
                            50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
                            51144, -- BANDS - hematology
                            50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
                            50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
                            50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
                            50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
                            -- 50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
                            50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
                            -- 50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
                            51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
                            -- 50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
                            51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
                            -- 50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
                            50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
                            51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
                            50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
                            -- 50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
                            51275, -- PTT | HEMATOLOGY | BLOOD | 474937
                            51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
                            51274, -- PT | HEMATOLOGY | BLOOD | 469090
                            50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
                            -- 50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
                            51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
                            51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
                            51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
                        )
                        AND valuenum IS NOT NULL
                        AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
                ),
                le_avg AS (
                    SELECT
                        le.subject_id,
                        le.charttime,
                        avg(
                            CASE
                                WHEN label = 'ANION GAP' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ANIONGAP,
                        avg(
                            CASE
                                WHEN label = 'ALBUMIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ALBUMIN,
                        avg(
                            CASE
                                WHEN label = 'BANDS' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BANDS,
                        avg(
                            CASE
                                WHEN label = 'BICARBONATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BICARBONATE,
                        avg(
                            CASE
                                WHEN label = 'BILIRUBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BILIRUBIN,
                        avg(
                            CASE
                                WHEN label = 'CREATININE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CREATININE,
                        avg(
                            CASE
                                WHEN label = 'CHLORIDE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CHLORIDE,
                        avg(
                            CASE
                                WHEN label = 'GLUCOSE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS GLUCOSE,
                        avg(
                            CASE
                                WHEN label = 'HEMATOCRIT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMATOCRIT,
                        avg(
                            CASE
                                WHEN label = 'HEMOGLOBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMOGLOBIN,
                        avg(
                            CASE
                                WHEN label = 'LACTATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS LACTATE,
                        avg(
                            CASE
                                WHEN label = 'PLATELET' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PLATELET,
                        avg(
                            CASE
                                WHEN label = 'POTASSIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS POTASSIUM,
                        avg(
                            CASE
                                WHEN label = 'PTT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PTT,
                        avg(
                            CASE
                                WHEN label = 'INR' THEN valuenum
                                ELSE NULL
                            END
                        ) AS INR,
                        avg(
                            CASE
                                WHEN label = 'PT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PT,
                        avg(
                            CASE
                                WHEN label = 'SODIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS SODIUM,
                        avg(
                            CASE
                                WHEN label = 'BUN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BUN,
                        avg(
                            CASE
                                WHEN label = 'WBC' THEN valuenum
                                ELSE NULL
                            END
                        ) AS WBC
                    FROM
                        le
                    GROUP BY
                        le.subject_id,
                        le.charttime
                )
                SELECT
                    iid.icustay_id,
                    adm.hadm_id,
                    le_avg.*
                FROM
                    le_avg
                    LEFT JOIN adm ON le_avg.subject_id = adm.subject_id
                    AND le_avg.charttime >= adm.data_start
                    AND le_avg.charttime < adm.data_end
                    LEFT JOIN iid_assign iid ON le_avg.subject_id = iid.subject_id
                    AND le_avg.charttime >= iid.data_start
                    AND le_avg.charttime < iid.data_end
                ORDER BY
                    le_avg.subject_id,
                    le_avg.charttime
            )
            SELECT
                *
            FROM
                pivoted_lab
        ) AS pl
        LEFT JOIN first_adm_offset f ON pl.icustay_id = f.id
        AND pl.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
mechvent AS (
    SELECT
        icustay_id,
        charttime -- case statement determining whether it is an instance of mech vent
,
        MAX(
            CASE
                WHEN itemid IS NULL
                OR value IS NULL THEN 0 -- can't have null values
                WHEN itemid = 720
                AND value != 'Other/Remarks' THEN 1 -- VentTypeRecorded
                WHEN itemid = 223848
                AND value != 'Other' THEN 1
                WHEN itemid = 223849 THEN 1 -- ventilator mode
                WHEN itemid = 467
                AND value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
                WHEN itemid IN(
                    445,
                    448,
                    449,
                    450,
                    1340,
                    1486,
                    1600,
                    224687 -- minute volume
,
                    639,
                    654,
                    681,
                    682,
                    683,
                    684,
                    224685,
                    224684,
                    224686 -- tidal volume
,
                    218,
                    436,
                    535,
                    444,
                    459,
                    224697,
                    224695,
                    224696,
                    224746,
                    224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
,
                    221,
                    1,
                    1211,
                    1655,
                    2000,
                    226873,
                    224738,
                    224419,
                    224750,
                    227187 -- Insp pressure
,
                    543 -- PlateauPressure
,
                    5865,
                    5866,
                    224707,
                    224709,
                    224705,
                    224706 -- APRV pressure
,
                    60,
                    437,
                    505,
                    506,
                    686,
                    220339,
                    224700 -- PEEP
,
                    3459 -- high pressure relief
,
                    501,
                    502,
                    503,
                    224702 -- PCV
,
                    223,
                    667,
                    668,
                    669,
                    670,
                    671,
                    672 -- TCPCV
,
                    224701 -- PSVlevel
                ) THEN 1
                ELSE 0
            END
        ) AS ventmarker
    FROM
        `physionet-data.mimiciii_clinical.chartevents` ce
        LEFT JOIN first_adm_offset f ON ce.icustay_id = f.id
        AND ce.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    WHERE
        ce.value IS NOT NULL -- exclude rows marked as error
        AND ce.error != 1 -- the below are settings used to indicate ventilation
        AND itemid IN(
            720,
            223849 -- vent mode
,
            223848 -- vent type
,
            445,
            448,
            449,
            450,
            1340,
            1486,
            1600,
            224687 -- minute volume
,
            639,
            654,
            681,
            682,
            683,
            684,
            224685,
            224684,
            224686 -- tidal volume
,
            218,
            436,
            535,
            444,
            224697,
            224695,
            224696,
            224746,
            224747 -- High/Low/Peak/Mean ("RespPressure")
,
            221,
            1,
            1211,
            1655,
            2000,
            226873,
            224738,
            224419,
            224750,
            227187 -- Insp pressure
,
            543 -- PlateauPressure
,
            5865,
            5866,
            224707,
            224709,
            224705,
            224706 -- APRV pressure
,
            60,
            437,
            505,
            506,
            686,
            220339,
            224700 -- PEEP
,
            3459 -- high pressure relief
,
            501,
            502,
            503,
            224702 -- PCV
,
            223,
            667,
            668,
            669,
            670,
            671,
            672 -- TCPCV
,
            224701 -- PSVlevel
            -- the below are settings used to indicate extubation
,
            640 -- extubated
            -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
,
            468 -- O2 Delivery Device#2
,
            469 -- O2 Delivery Mode
,
            470 -- O2 Flow (lpm)
,
            471 -- O2 Flow (lpm) #2
,
            227287 -- O2 Flow (additional cannula)
,
            226732 -- O2 Delivery Device(s)
,
            223834 -- O2 Flow
            -- used in both oxygen + vent calculation
,
            467 -- O2 Delivery Device
        )
    GROUP BY
        icustay_id,
        charttime
),
sepsis AS (
    -- ICD-9 codes for Angus criteria of sepsis
    SELECT
        hadm_id,
        MAX(
            CASE
                WHEN SUBSTR(icd9_code, 1, 3) IN (
                    '001','002','003','004','005',
                    '008','009','010','011','012',
                    '013','014','015','016','017',
                    '018','020','021','022','023',
                    '024','025','026','027','030',
                    '031','032','033','034','035',
                    '036','037','038','039','040',
                    '041','090','091','092','093',
                    '094','095','096','097','098',
                    '100','101','102','103','104',
                    '110','111','112','114','115',
                    '116','117','118','320','322',
                    '324','325','420','421','451',
                    '461','462','463','464','465',
                    '481','482','485','486','494',
                    '510','513','540','541','542',
                    '566','567','590','597','601',
                    '614','615','616','681','682',
                    '683','686','730'
                ) THEN 1
                WHEN SUBSTR(icd9_code, 1, 4) IN (
                    '5695','5720','5721','5750',
                    '5990','7110','7907','9966',
                    '9985','9993'
                ) THEN 1
                WHEN SUBSTR(icd9_code, 1, 5) IN (
                    '49121','56201','56203',
                    '56211','56213','56983'
                ) THEN 1
                ELSE 0
            END
        ) AS sepsisflag
    FROM
        `physionet-data.mimiciii_clinical.diagnoses_icd`
    GROUP BY
        hadm_id
),
vasopressor_duration AS (
    SELECT
        *
    FROM
        `physionet-data.mimiciii_derived.vasopressordurations`
),
premorbid AS (
    WITH icd AS (
        SELECT
            hadm_id,
            seq_num,
            CAST(icd9_code AS STRING) AS icd9_code
        FROM
            `physionet-data.mimiciii_clinical.diagnoses_icd`
    ),
    eliflg AS (
        SELECT
            hadm_id,
            seq_num,
            icd9_code -- note that these codes will seem incomplete at first
            -- for example, CHF is missing a lot of codes referenced in the literature (402.11, 402.91, etc)
            -- these codes are captured by hypertension flags instead
            -- later there are some complicated rules which confirm/reject those codes as CHF
,
            CASE
                WHEN SUBSTR(icd9_code, 1, 3) IN(
                    '410', --Acute myocardial infarction
                    '411', --Other acute and subacute forms of ischemic heart disease
                    '412', --Old myocardial infarction
                    '413', --Angina pectoris
                    '414'  --Other forms of chronic ischemic heart disease
                ) THEN 1
            END AS IHD,
            CASE
                WHEN icd9_code = '39891' THEN 1
                WHEN icd9_code BETWEEN '4280 '
                AND '4289 ' THEN 1
            END AS CHF
            /* Congestive heart failure */
            -- cardiac arrhythmias is removed in up to date versions
,
            CASE
                WHEN icd9_code BETWEEN '4400 '
                AND '4409 ' THEN 1
                WHEN icd9_code BETWEEN '44100'
                AND '4419 ' THEN 1
                WHEN icd9_code BETWEEN '4420 '
                AND '4429 ' THEN 1
                WHEN icd9_code BETWEEN '4431 '
                AND '4439 ' THEN 1
                WHEN icd9_code BETWEEN '44421'
                AND '44422' THEN 1
                WHEN icd9_code = '4471 ' THEN 1
                WHEN icd9_code = '449  ' THEN 1
                WHEN icd9_code = '5571 ' THEN 1
                WHEN icd9_code = '5579 ' THEN 1
                WHEN icd9_code = 'V434 ' THEN 1
            END AS PERIVASC
            /* Peripheral vascular disorder */
,
            CASE
                WHEN icd9_code = '40200' THEN 1
                WHEN icd9_code = '40210' THEN 1
                WHEN icd9_code = '40290' THEN 1
                WHEN icd9_code = '40509' THEN 1
                WHEN icd9_code = '40519' THEN 1
                WHEN icd9_code = '40599' THEN 1
            END AS HTNWOCHF
            /* Hypertensive heart disease without heart failure */
,
            CASE
                WHEN icd9_code = '40201' THEN 1
                WHEN icd9_code = '40211' THEN 1
                WHEN icd9_code = '40291' THEN 1
            END AS HTNWCHF
            /* Hypertensive heart disease with heart failure */
,
            CASE
                WHEN icd9_code = '40301' THEN 1
                WHEN icd9_code = '40311' THEN 1
                WHEN icd9_code = '40391' THEN 1
            END AS HRENWRF
            /* Hypertensive renal disease with renal failure */
,
            CASE
                WHEN icd9_code = '40401' THEN 1
                WHEN icd9_code = '40411' THEN 1
                WHEN icd9_code = '40491' THEN 1
            END AS HHRWCHF
            /* Hypertensive heart and renal disease with heart failure */
,
            CASE
                WHEN icd9_code = '40402' THEN 1
                WHEN icd9_code = '40412' THEN 1
                WHEN icd9_code = '40492' THEN 1
            END AS HHRWRF
            /* Hypertensive heart and renal disease with renal failure */
,
            CASE
                WHEN icd9_code = '40403' THEN 1
                WHEN icd9_code = '40413' THEN 1
                WHEN icd9_code = '40493' THEN 1
            END AS HHRWHRF
            /* Hypertensive heart and renal disease with heart and renal failure */
,
            CASE
                WHEN icd9_code = '585  ' THEN 1 -- discontinued code
                WHEN icd9_code = '5853 ' THEN 1
                WHEN icd9_code = '5854 ' THEN 1
                WHEN icd9_code = '5855 ' THEN 1
                WHEN icd9_code = '5856 ' THEN 1
                WHEN icd9_code = '5859 ' THEN 1
                WHEN icd9_code = '586  ' THEN 1
                WHEN icd9_code = 'V420 ' THEN 1
                WHEN icd9_code = 'V451 ' THEN 1
                WHEN icd9_code BETWEEN 'V560 '
                AND 'V5632' THEN 1
                WHEN icd9_code = 'V568 ' THEN 1
                WHEN icd9_code BETWEEN 'V4511'
                AND 'V4512' THEN 1
            END AS RENLFAIL
            /* Renal failure */
        FROM
            icd
    ),
    eligrp AS (
        SELECT
            hadm_id,
            MAX(ihd) AS ihd,
            MAX(chf) AS chf,
            MAX(perivasc) AS perivasc,
            MAX(htnwchf) AS htnwchf,
            MAX(hrenwrf) AS hrenwrf,
            MAX(hhrwchf) AS hhrwchf,
            MAX(hhrwrf) AS hhrwrf,
            MAX(hhrwhrf) AS hhrwhrf,
            MAX(renlfail) AS renlfail
        FROM
            eliflg
        GROUP BY
            hadm_id
    )
    SELECT
        adm.subject_id,
        adm.hadm_id,
        CASE
            WHEN ihd = 1 THEN 1
            ELSE 0
        END AS ISCHEMIC_HEART_DISEASE,
        CASE
            WHEN chf = 1 THEN 1
            WHEN htnwchf = 1 THEN 1
            WHEN hhrwchf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS CONGESTIVE_HEART_FAILURE,
        CASE
            WHEN perivasc = 1 THEN 1
            ELSE 0
        END AS PERIPHERAL_VASCULAR,
        CASE
            WHEN renlfail = 1 THEN 1
            WHEN hrenwrf = 1 THEN 1
            WHEN hhrwrf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS RENAL_FAILURE
    FROM
        `physionet-data.mimiciii_clinical.admissions` adm
        LEFT JOIN eligrp eli ON adm.hadm_id = eli.hadm_id
    ORDER BY
        adm.hadm_id
),
surgflag AS (
    SELECT
        adm.hadm_id,
        CASE
            WHEN LOWER(curr_service) LIKE '%surg%' THEN 1
            ELSE 0
        END AS surgical,
        ROW_NUMBER() OVER(
            PARTITION BY adm.HADM_ID
            ORDER BY
                TRANSFERTIME
        ) AS serviceOrder
    FROM
        `physionet-data.mimiciii_clinical.admissions` adm
        LEFT JOIN `physionet-data.mimiciii_clinical.services` se ON adm.hadm_id = se.hadm_id
),
tx_raw AS (
    -- BLOOD TRANSFUSED in mL
    WITH carevue AS (
        SELECT
            cv.icustay_id,
            cv.charttime,
            -- carevue always has units in millilitres (or null)
            ROUND(cv.amount) AS amount
        FROM
            `physionet-data.mimiciii_clinical.inputevents_cv` cv
        WHERE
            cv.itemid IN(
                30001,  -- Packed RBC's
                30002,  -- Whole Blood
                30004,  -- Washed PRBC's
                30104,  -- OR Packed RBC's
                30179,  -- PRBC's
                42588,  -- VICU PRBC
                42239,  -- CC7 PRBC
                42186,	-- Pre admit PRBC
                42239,	-- CC7 PRBC
                42324,	-- er prbc
                42588,	-- VICU PRBC
                42740,	-- prbc in er
                43010,	-- packed cells pacu
                45020,	-- RBC waste
                46124,	-- er in prbc
                46407,	-- ED PRBC
                46612	  -- E.R. prbc
            )
            AND amount > 100 --do not count any transfusions events <100mls
    )
    SELECT
        cv.icustay_id,
        charttime,
        sum(amount) AS rbc_volume,
    FROM
        carevue cv
        LEFT JOIN first_adm_offset f ON cv.icustay_id = f.id
        AND cv.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    GROUP BY
        cv.icustay_id,
        cv.charttime
),
tx AS (
    SELECT
        icustay_id,
        TIMESTAMP_SECONDS(60 * DIV(UNIX_SECONDS(TIMESTAMP(charttime)), 60)) AS episode,
        --some samples have duplicate readings every 15 minutes - we take an hourly window and average the volume across this window
        --doing so reduces errors significantly
        MIN(charttime) AS charttime,
        --get the first timestamp of each window
        AVG(rbc_volume) AS rbc_volume
    FROM tx_raw
    GROUP BY icustay_id, episode
    ORDER BY icustay_id, episode
),
tx_counts AS (
    SELECT
        icustay_id,
        COUNT(charttime) AS tx_episodes,
        SUM(rbc_volume) AS rbc_volume,
        CEIL(SUM(rbc_volume) / 350) AS rbc_units
    FROM tx
    WHERE
        icustay_id IS NOT NULL
    GROUP BY
        icustay_id
),
pivoted_hemoglobin AS (
    SELECT
        icustay_id,
        charttime,
        hemoglobin
    FROM
        pivoted_lab
    WHERE
        icustay_id IS NOT NULL
        AND charttime IS NOT NULL
        AND hemoglobin IS NOT NULL
    ORDER BY
        icustay_id
)
, tx_hgb_paired AS (
        SELECT
            w.icustay_id,
            w.last_hb_in_window,
            MIN(p.hemoglobin) AS hemoglobin,
            w.tx_offset,
            w.rbc_volume,
            DATETIME_DIFF(w.tx_offset, w.last_hb_in_window, SECOND) AS delta_tx_hgb_seconds,
        FROM
            (
                SELECT
                    p.icustay_id,
                    t.charttime AS tx_offset,
                    t.rbc_volume,
                    MAX(p.charttime) AS last_hb_in_window,
                    p.hemoglobin
                FROM tx t --`arctic-welder-307905.blood_transfusion.tx` t
                    INNER JOIN pivoted_hemoglobin p ON p.icustay_id = t.icustay_id
                    AND p.charttime BETWEEN DATETIME_SUB(t.charttime, INTERVAL 6 HOUR)
                    AND t.charttime --we search the 6 hours prior to transfusion for minimum hemoglobin levels associated with that transfusion
                GROUP BY
                    p.icustay_id,
                    t.charttime,
                    t.rbc_volume,
                    p.hemoglobin
                ORDER BY
                    icustay_id,
                    last_hb_in_window
            ) w
            INNER JOIN pivoted_hemoglobin p ON p.icustay_id = w.icustay_id
            AND p.charttime = w.last_hb_in_window
        GROUP BY
            w.icustay_id,
            w.last_hb_in_window,
            w.tx_offset,
            w.rbc_volume
        ORDER BY
            icustay_id,
             last_hb_in_window,
             tx_offset
)
, tx_hgb_filtered AS (
        SELECT
            *,
            ROW_NUMBER() OVER(x) AS event_number
        FROM tx_hgb_paired
        WINDOW x AS(
                PARTITION BY icustay_id
                ORDER BY tx_offset
            )
        ORDER BY icustay_id

)
--hr_tx_from_icu_intime AS
--(
  --select ih.icustay_id, ie.hadm_id
  --, hr,ie.INTIME
  -- start/endtime can be used to filter to values within this hour
  --, DATETIME_SUB(ih.endtime, INTERVAL '1' HOUR) AS starttime
  --, ih.endtime
  --from `physionet-data.mimiciii_derived.icustay_hours` ih
  --INNER JOIN `physionet-data.mimiciii_clinical.icustays` ie
    --ON ih.icustay_id = ie.icustay_id
--),
--hours_from_icu_in_time AS(
--select * from hr_tx_from_icu_intime
--order by icustay_id, hr)

, closest_tx_hb_pairing AS (SELECT icustay_id,
      ROW_NUMBER() OVER(PARTITION BY icustay_id ORDER BY tx_offset) AS tx_episode,
     last_hb_in_window, hemoglobin,
      tx_offset, rbc_volume,
      delta_tx_hgb_seconds/60 AS delta_offset_minutes
    FROM tx_hgb_filtered
    ORDER BY icustay_id, event_number),

closest_tx_hb_pairing_icu_time AS (SELECT a.*,b.INTIME,b.OUTTIME,ROUND((rbc_volume/ 350),0) AS rbc_units
FROM closest_tx_hb_pairing  a
INNER JOIN  `physionet-data.mimiciii_clinical.icustays` b
ON a.icustay_id= b.icustay_id
AND a.tx_offset BETWEEN b.INTIME and b.OUTTIME
ORDER BY a.icustay_id)

SELECT * FROM closest_tx_hb_pairing_icu_time
""").to_dataframe()

In [ ]:
#Repeat the same steps for mimic 3
df= tx_mimic3_carevue.copy()
df.sort_values(by = ['icustay_id', 'tx_offset','delta_offset_minutes'], ascending = [True, True,True], inplace=True)
df.drop_duplicates(subset=['icustay_id'], keep= 'first', inplace=True)
mimic3_carevue_1rbc = df.loc[df['rbc_units'] == 1.0]
# rename icustay to stay_id
mimic3_carevue_1rbc=mimic3_carevue_1rbc.rename(columns={"icustay_id": "stay_id"})
mimic3_carevue_1rbc.head()

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units
0,200009,1,2189-11-30 18:29:00,10.5,2189-11-30 23:00:00,375.0,271.0,2189-11-30 10:34:32,2189-12-02 14:17:37,1.0
4,200041,1,2129-04-01 19:15:00,10.1,2129-04-02 00:00:00,375.0,285.0,2129-04-01 18:43:39,2129-04-03 22:56:02,1.0
6,200059,2,2198-02-11 01:34:00,9.9,2198-02-11 01:45:00,375.0,11.0,2198-02-10 06:05:00,2198-02-23 14:41:10,1.0
13,200065,1,2120-03-23 12:25:00,7.5,2120-03-23 12:30:00,375.0,5.0,2120-03-22 22:30:45,2120-04-05 15:33:42,1.0
30,200077,1,2163-04-17 21:30:00,8.5,2163-04-18 02:15:00,375.0,285.0,2163-04-08 17:36:00,2163-04-24 14:40:00,1.0


# Sepsis MIMIC 4

In [ ]:
sepsis_mimic4 = client.query("""
SELECT *
FROM `physionet-data.mimic_derived.sepsis3`
""").to_dataframe()

In [ ]:
#Keep only septic patients
tx_mimic4_septic_1rbc=pd.merge(treatment_mimic4_1rbc,sepsis_mimic4,how='inner',on='stay_id')
#Sepsis-3 cohort for treatment group mimic4
tx_mimic4_septic_1rbc[treatment_mimic4_1rbc.columns]

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units
0,30001446,1,2186-04-12 02:00:00,7.9,2186-04-12 04:28:00,375.0,148.0,2186-04-12 03:49:00,2186-04-13 19:45:47,1.0
1,30001555,1,2177-09-27 11:50:00,7.6,2177-09-27 16:00:00,350.0,250.0,2177-09-27 11:23:13,2177-09-28 18:26:00,1.0
2,30003749,2,2120-11-05 16:30:00,6.3,2120-11-05 17:48:00,375.0,78.0,2120-11-05 16:03:00,2120-11-07 08:19:44,1.0
3,30004018,1,2158-12-31 05:14:00,6.6,2158-12-31 10:02:00,375.0,288.0,2158-12-27 21:32:00,2159-01-12 23:47:50,1.0
4,30004811,1,2139-10-06 18:03:00,8.8,2139-10-06 19:03:00,375.0,60.0,2139-10-06 10:40:29,2139-10-08 15:56:05,1.0
...,...,...,...,...,...,...,...,...,...,...
8081,39990055,1,2158-05-03 01:54:00,6.9,2158-05-03 03:39:00,276.0,105.0,2158-05-01 23:32:15,2158-05-04 18:52:13,1.0
8082,39992167,1,2114-06-10 21:12:00,6.8,2114-06-10 21:54:00,350.0,42.0,2114-06-10 19:00:00,2114-06-20 20:09:43,1.0
8083,39995213,1,2127-12-14 03:48:00,6.9,2127-12-14 04:56:00,350.0,68.0,2127-12-11 18:03:43,2127-12-17 21:07:04,1.0
8084,39996044,1,2164-12-11 07:58:00,5.7,2164-12-11 12:07:00,304.0,249.0,2164-12-10 17:50:47,2164-12-17 18:23:16,1.0


# Sepsis MIMIC 3

In [ ]:
sepsis_mimiciii= client.query("""
--- Exclusion refers to exclusion criteria set by the original code. Refer to github for further information
-- patients <16 years old
-- take into account only first icu addmission

WITH sepsis3_cohort AS (with serv as
(
select hadm_id, curr_service
    , ROW_NUMBER() over (partition by hadm_id order by transfertime) as rn
    from `physionet-data.mimiciii_clinical.services`
)
, t1 as
(
select ie.icustay_id, ie.hadm_id
    , ie.intime, ie.outtime
    ---, round((cast(adm.admittime as date) - cast(pat.dob as date)) / 365.242, 4) as age
    , ROUND(TIMESTAMP_DIFF(adm.admittime,pat.dob, DAY)/ 365.242,4) AS age
    , pat.gender
    , adm.ethnicity
    , ie.dbsource
    -- used to get first ICUSTAY_ID
    , ROW_NUMBER() over (partition by ie.subject_id order by intime) as rn

    -- exclusions
    , s.curr_service as first_service
    , adm.HAS_CHARTEVENTS_DATA

    -- suspicion of infection using POE
    , case when spoe.suspected_infection_time is not null then 1 else 0 end
        as suspected_of_infection_poe
    , spoe.suspected_infection_time as suspected_infection_time_poe
    ,TIMESTAMP_DIFF(ie.intime,spoe.suspected_infection_time,SECOND)/ 60.0 / 60.0 / 24.0 as suspected_infection_time_poe_days
    , spoe.specimen as specimen_poe
    , spoe.positiveculture as positiveculture_poe
    , spoe.antibiotic_time as antibiotic_time_poe

from `physionet-data.mimiciii_clinical.icustays` ie
inner join `physionet-data.mimiciii_clinical.admissions` adm
    on ie.hadm_id = adm.hadm_id
inner join `physionet-data.mimiciii_clinical.patients` pat
    on ie.subject_id = pat.subject_id
left join serv s
    on ie.hadm_id = s.hadm_id
    and s.rn = 1
left join `physionet-data.mimiciii_derived.suspinfect_poe` spoe
  on ie.icustay_id = spoe.icustay_id
)
select
    t1.hadm_id, t1.icustay_id
  , t1.intime, t1.outtime

  -- set de-identified ages to median of 91.4
  , case when age > 89 then 91.4 else age end as age
  , gender
  , ethnicity
  , first_service
  , dbsource

  -- suspicion using POE
  , suspected_of_infection_poe
  , suspected_infection_time_poe
  , suspected_infection_time_poe_days
  , specimen_poe
  , positiveculture_poe
  , antibiotic_time_poe

  -- exclusions
  , case when t1.rn = 1 then 0 else 1 end as exclusion_secondarystay
  , case when t1.age <= 16 then 1 else 0 end as exclusion_nonadult
  , case when t1.first_service in ('CSURG','VSURG','TSURG') then 1 else 0 end as exclusion_csurg
  , case when t1.dbsource != 'metavision' then 1 else 0 end as exclusion_carevue
  , case when t1.suspected_infection_time_poe is not null
          and t1.suspected_infection_time_poe < (t1.intime-interval '1' day) then 1
      else 0 end as exclusion_early_suspicion
  , case when t1.suspected_infection_time_poe is not null
          and t1.suspected_infection_time_poe > (t1.intime+interval '1' day) then 1
      else 0 end as exclusion_late_suspicion
  , case when t1.HAS_CHARTEVENTS_DATA = 0 then 1
         when t1.intime is null then 1
         when t1.outtime is null then 1
      else 0 end as exclusion_bad_data
  -- , case when t1.suspected_of_infection = 0 then 1 else 0 end as exclusion_suspicion

  -- the above flags are used to summarize patients excluded
  -- below flag is used to actually exclude patients in future queries
  , case when
             t1.rn != 1
          or t1.age <= 16
          or t1.first_service in ('CSURG','VSURG','TSURG')
          or t1.HAS_CHARTEVENTS_DATA = 0
          or t1.intime is null
          or t1.outtime is null
          or t1.dbsource != 'metavision'
          or (
                  t1.suspected_infection_time_poe is not null
              and t1.suspected_infection_time_poe < (t1.intime-interval '1' day)
            )
          or (
                  t1.suspected_infection_time_poe is not null
              and t1.suspected_infection_time_poe > (t1.intime+interval '1' day)
            )
          -- or t1.suspected_of_infection = 0
            then 1
        else 0 end as excluded
from t1
order by t1.icustay_id)

select co.icustay_id, co.hadm_id
-- exclusion criteria
    , co.excluded
    , co.intime, co.outtime
    , ie.dbsource
-- suspicion POE
    , co.suspected_infection_time_poe
    , co.suspected_infection_time_poe_days
    , co.specimen_poe
    , co.positiveculture_poe
    , co.antibiotic_time_poe
-- blood culture on admission
    , bc.charttime as blood_culture_time
    , bc.positiveculture as blood_culture_positive
    , co.age
    , co.gender
    , case when co.gender = 'M' then 1 else 0 end as is_male
    , co.ethnicity
-- ethnicity flags
    , case when co.ethnicity in
    (
'WHITE' --  40996
       , 'WHITE - RUSSIAN' --    164
       , 'WHITE - OTHER EUROPEAN' --     81
       , 'WHITE - BRAZILIAN' --     59
       , 'WHITE - EASTERN EUROPEAN' --     25
    ) then 1 else 0 end as race_white
    , case when co.ethnicity in
    (
'BLACK/AFRICAN AMERICAN' --   5440
        , 'BLACK/CAPE VERDEAN' --    200
        , 'BLACK/HAITIAN' --    101
        , 'BLACK/AFRICAN' --     44
        , 'CARIBBEAN ISLAND' --      9
    ) then 1 else 0 end as race_black
    , case when co.ethnicity in
    (
'HISPANIC OR LATINO' --   1696
    , 'HISPANIC/LATINO - PUERTO RICAN' --    232
    , 'HISPANIC/LATINO - DOMINICAN' --     78
    , 'HISPANIC/LATINO - GUATEMALAN' --     40
    , 'HISPANIC/LATINO - CUBAN' --     24
    , 'HISPANIC/LATINO - SALVADORAN' --     19
    , 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)' --     13
    , 'HISPANIC/LATINO - MEXICAN' --     13
    , 'HISPANIC/LATINO - COLOMBIAN' --      9
    , 'HISPANIC/LATINO - HONDURAN' --      4
  ) then 1 else 0 end as race_hispanic
  , case when co.ethnicity not in
  (
'WHITE' --  40996
    , 'WHITE - RUSSIAN' --    164
    , 'WHITE - OTHER EUROPEAN' --     81
    , 'WHITE - BRAZILIAN' --     59
    , 'WHITE - EASTERN EUROPEAN' --     25
    , 'BLACK/AFRICAN AMERICAN' --   5440
    , 'BLACK/CAPE VERDEAN' --    200
    , 'BLACK/HAITIAN' --    101
    , 'BLACK/AFRICAN' --     44
    , 'CARIBBEAN ISLAND' --      9
    , 'HISPANIC OR LATINO' --   1696
    , 'HISPANIC/LATINO - PUERTO RICAN' --    232
    , 'HISPANIC/LATINO - DOMINICAN' --     78
    , 'HISPANIC/LATINO - GUATEMALAN' --     40
    , 'HISPANIC/LATINO - CUBAN' --     24
    , 'HISPANIC/LATINO - SALVADORAN' --     19
    , 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)' --     13
    , 'HISPANIC/LATINO - MEXICAN' --     13
    , 'HISPANIC/LATINO - COLOMBIAN' --      9
    , 'HISPANIC/LATINO - HONDURAN' --      4
  ) then 1 else 0 end as race_other
-- other races
-- , 'ASIAN' --   1509
-- , 'ASIAN - CHINESE' --    277
-- , 'ASIAN - ASIAN INDIAN' --     85
-- , 'ASIAN - VIETNAMESE' --     53
-- , 'ASIAN - FILIPINO' --     25
-- , 'ASIAN - CAMBODIAN' --     17
-- , 'ASIAN - OTHER' --     17
-- , 'ASIAN - KOREAN' --     13
-- , 'ASIAN - JAPANESE' --      7
-- , 'ASIAN - THAI' --      4
--
-- , 'UNKNOWN/NOT SPECIFIED' --   4523
-- , 'OTHER' --   1512
-- , 'UNABLE TO OBTAIN' --    814
-- , 'PATIENT DECLINED TO ANSWER' --    559
-- , 'MULTI RACE ETHNICITY' --    130
-- , 'PORTUGUESE' --     61
-- , 'AMERICAN INDIAN/ALASKA NATIVE' --     51
-- , 'MIDDLE EASTERN' --     43
-- , 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' --     18
-- , 'SOUTH AMERICAN' --      8
-- , 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' --      3
    , eli.metastatic_cancer
    , case when eli.diabetes_uncomplicated = 1
or eli.diabetes_complicated = 1
                then 1
        else 0 end as diabetes
    , ht.Height
    , wt.Weight
    , wt.Weight / (ht.Height/100*ht.Height/100) as bmi
-- service type on hospital admission
    , co.first_service
-- outcomes
    , adm.HOSPITAL_EXPIRE_FLAG
    , case when pat.dod <=DATETIME_ADD(co.intime, INTERVAL 30 DAY) then 1 else 0 end
as THIRTYDAY_EXPIRE_FLAG
    , ie.los as icu_los
    , TIMESTAMP_DIFF(adm.dischtime, adm.admittime, SECOND)/60.0/60.0/24.0 as hosp_los
-- sepsis flags
    , a.angus as sepsis_angus
    , m.sepsis as sepsis_martin
    , es.sepsis as sepsis_explicit
    , es.septic_shock as septic_shock_explicit
    , es.severe_sepsis as severe_sepsis_explicit
    , nqf.sepsis as sepsis_nqf
    , cdc.sepsis as sepsis_cdc
    , cdc.sepsis_simple as sepsis_cdc_simple
-- in-hospital mortality score (van Walraven et al.)
    ,   CONGESTIVE_HEART_FAILURE    *(4)    + CARDIAC_ARRHYTHMIAS   *(4) +
        VALVULAR_DISEASE            *(-3)   + PULMONARY_CIRCULATION *(0) +
        PERIPHERAL_VASCULAR         *(0)    + HYPERTENSION*(-1) + PARALYSIS*(0) +
        OTHER_NEUROLOGICAL          *(7)    + CHRONIC_PULMONARY*(0) +
        DIABETES_UNCOMPLICATED      *(-1)   + DIABETES_COMPLICATED*(-4) +
        HYPOTHYROIDISM              *(0)    + RENAL_FAILURE*(3) + LIVER_DISEASE*(4) +
        PEPTIC_ULCER                *(-9)   + AIDS*(0) + LYMPHOMA*(7) +
        METASTATIC_CANCER           *(9)    + SOLID_TUMOR*(0) + RHEUMATOID_ARTHRITIS*(0) +
        COAGULOPATHY                *(3)    + OBESITY*(-5) +
        WEIGHT_LOSS                 *(4)    + FLUID_ELECTROLYTE         *(6) +
        BLOOD_LOSS_ANEMIA           *(0)    + DEFICIENCY_ANEMIAS      *(-4) +
        ALCOHOL_ABUSE               *(0)    + DRUG_ABUSE*(-6) +
        PSYCHOSES                   *(-5)   + DEPRESSION*(-8)
as elixhauser_hospital
    , case when vent.starttime is not null then 1 else 0 end as vent
    , so.sofa as sofa
    , lo.lods as lods
    , si.sirs as sirs
    , qs.qsofa as qsofa
-- subcomponents for qSOFA
    , qs.SysBP_score as qsofa_sysbp_score
    , qs.GCS_score as qsofa_gcs_score
    , qs.RespRate_score as qsofa_resprate_score
from sepsis3_cohort co
inner join `physionet-data.mimiciii_clinical.icustays` ie
 on co.icustay_id = ie.icustay_id
inner join `physionet-data.mimiciii_clinical.admissions` adm
 on ie.hadm_id = adm.hadm_id
inner join `physionet-data.mimiciii_clinical.patients`  pat
 on ie.subject_id = pat.subject_id
left join  `physionet-data.mimiciii_derived.elixhauser_ahrq_v37` eli
 on ie.hadm_id = eli.hadm_id
left join `physionet-data.mimiciii_derived.heightfirstday` ht
 on ie.icustay_id = ht.icustay_id
left join `physionet-data.mimiciii_derived.weight_first_day` wt
 on ie.icustay_id = wt.icustay_id
left join `physionet-data.mimiciii_derived.angus_sepsis` a
 on ie.hadm_id = a.hadm_id
left join `physionet-data.mimiciii_derived.martin_sepsis` m
 on ie.hadm_id = m.hadm_id
left join `physionet-data.mimiciii_derived.explicit_sepsis` es
 on ie.hadm_id = es.hadm_id
left join `physionet-data.mimiciii_derived.sepsis_nqf_0500` nqf
 on ie.icustay_id = nqf.icustay_id
left join `physionet-data.mimiciii_derived.sepsis_cdc_surveillance`cdc
 on ie.icustay_id = cdc.icustay_id
left join `physionet-data.mimiciii_derived.blood_culture_icu_admit` bc
 on ie.icustay_id = bc.icustay_id

left join ( select icustay_id, min(starttime) as starttime
   from `physionet-data.mimiciii_derived.ventdurations`
   group by icustay_id) vent
on co.icustay_id = vent.icustay_id
and vent.starttime >= DATETIME_ADD(co.intime, INTERVAL 4 HOUR)
and vent.starttime <= DATETIME_SUB(co.intime, INTERVAL 1 DAY)
left join `physionet-data.mimiciii_derived.sofa` so
 on co.icustay_id = so.icustay_id
left join `physionet-data.mimiciii_derived.sirs` si
 on co.icustay_id = si.icustay_id
left join  `physionet-data.mimiciii_derived.lods`  lo
 on co.icustay_id = lo.icustay_id
left join `physionet-data.mimiciii_derived.qsofa` qs
 on co.icustay_id = qs.icustay_id
 order by co.icustay_id
""").to_dataframe()

In [ ]:
#Insert a new column about suspicion of infection
sepsis_mimiciii['suspicion_poe'] = (~sepsis_mimiciii['suspected_infection_time_poe_days'].isnull())
#If poe is true and sofa score is equal or bigger than 2 then we have sepsis
sepsis_mimiciii['sepsis-3'] = ((sepsis_mimiciii['suspicion_poe']==1) & (sepsis_mimiciii['sofa']>=2)).astype(int)
sepsis_mimiciii= sepsis_mimiciii[sepsis_mimiciii['sepsis-3']==1]
sepsis_mimiciii.rename(columns = {'icustay_id':'stay_id'}, inplace = True)
tx_mimic3_carevue_septic_1rbc=pd.merge(mimic3_carevue_1rbc,sepsis_mimiciii,how='inner',on='stay_id')
#Sepsis-3 cohort for treatment group mimic3
tx_mimic3_carevue_septic_1rbc.head()

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units,...,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score,suspicion_poe,sepsis-3
0,200112,1,2140-07-23 08:52:00,7.9,2140-07-23 09:00:00,375.0,8.0,2140-07-23 05:40:40,2140-07-24 13:48:03,1.0,...,0,3,4,3,1,0.0,0.0,1.0,True,1
1,200492,1,2102-06-12 01:55:00,7.3,2102-06-12 06:00:00,400.0,245.0,2102-06-12 03:51:36,2102-06-14 14:40:25,1.0,...,0,4,7,4,3,1.0,1.0,1.0,True,1
2,200535,1,2184-11-10 05:40:00,7.4,2184-11-10 11:15:00,375.0,335.0,2184-11-10 07:40:35,2184-11-11 18:39:45,1.0,...,0,5,3,2,2,1.0,1.0,0.0,True,1
3,200660,1,2193-07-09 04:00:00,9.5,2193-07-09 04:30:00,375.0,30.0,2193-07-02 02:19:53,2193-07-10 00:43:35,1.0,...,0,6,6,4,2,1.0,0.0,1.0,True,1
4,200793,1,2139-02-11 13:33:00,8.8,2139-02-11 16:00:00,300.0,147.0,2139-02-10 20:03:07,2139-02-13 21:39:22,1.0,...,0,10,8,4,2,1.0,0.0,1.0,True,1


# Joining MIMIC 4 (METAVISION) with Mimic 3 (carevue) - FULL COHORT TREATMENT

In [ ]:
treatment_group = pd.concat([mimic3_carevue_1rbc,treatment_mimic4_1rbc])
#All patient that received 1 blood sac (1st icustay)
treatment_group.head()

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units
0,200009,1,2189-11-30 18:29:00,10.5,2189-11-30 23:00:00,375.0,271.0,2189-11-30 10:34:32,2189-12-02 14:17:37,1.0
4,200041,1,2129-04-01 19:15:00,10.1,2129-04-02 00:00:00,375.0,285.0,2129-04-01 18:43:39,2129-04-03 22:56:02,1.0
6,200059,2,2198-02-11 01:34:00,9.9,2198-02-11 01:45:00,375.0,11.0,2198-02-10 06:05:00,2198-02-23 14:41:10,1.0
13,200065,1,2120-03-23 12:25:00,7.5,2120-03-23 12:30:00,375.0,5.0,2120-03-22 22:30:45,2120-04-05 15:33:42,1.0
30,200077,1,2163-04-17 21:30:00,8.5,2163-04-18 02:15:00,375.0,285.0,2163-04-08 17:36:00,2163-04-24 14:40:00,1.0


# SEPTIC - Joining MIMIC 4 (METAVISION) with Mimic 3 (carevue) for septic patients - FULL SEPTIC COHORT TREATMENT

In [ ]:
treatment_group_septic = pd.concat([tx_mimic4_septic_1rbc,tx_mimic3_carevue_septic_1rbc])
#All septic patients that received 1 blood sac (1st icustay)
treatment_group_septic.head()

,stay_id,tx_episode,last_hb_in_window,hemoglobin,tx_offset,rbc_volume,delta_offset_minutes,INTIME,OUTTIME,rbc_units,...,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score,suspicion_poe,sepsis-3
0,30001446,1,2186-04-12 02:00:00,7.9,2186-04-12 04:28:00,375.0,148.0,2186-04-12 03:49:00,2186-04-13 19:45:47,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30001555,1,2177-09-27 11:50:00,7.6,2177-09-27 16:00:00,350.0,250.0,2177-09-27 11:23:13,2177-09-28 18:26:00,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30003749,2,2120-11-05 16:30:00,6.3,2120-11-05 17:48:00,375.0,78.0,2120-11-05 16:03:00,2120-11-07 08:19:44,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30004018,1,2158-12-31 05:14:00,6.6,2158-12-31 10:02:00,375.0,288.0,2158-12-27 21:32:00,2159-01-12 23:47:50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30004811,1,2139-10-06 18:03:00,8.8,2139-10-06 19:03:00,375.0,60.0,2139-10-06 10:40:29,2139-10-08 15:56:05,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# MIMIC 4 Control

In [ ]:
control_mimic4 = client.query("""
WITH all_criteria AS (SELECT DISTINCT(stay_id)
FROM `physionet-data.mimic_derived.sofa`
WHERE stay_id IN(SELECT DISTINCT(stay_id)
                    FROM `physionet-data.mimic_derived.icustay_detail`
                    WHERE icustay_seq = 1
                    AND first_icu_stay= TRUE
                    AND (admission_age BETWEEN 18 AND 89)
  AND stay_id NOT IN(SELECT DISTINCT(i.stay_id)
        FROM `physionet-data.mimic_hosp.diagnoses_icd` d
        INNER JOIN `physionet-data.mimic_icu.icustays` i
            ON d.hadm_id=i.hadm_id
        INNER JOIN `physionet-data.mimic_derived.icustay_detail` a
            ON d.HADM_ID=a.hadm_id
        WHERE icd_code IN (
                --ref: https://www.bmj.com/content/bmj/suppl/2015/02/03/bmj.h246.DC1/junm020747.ww1_default.pdf
                --intracranial bleeding
                '430', --Subarachnoid
                '431', --intracerebral
                '4320', --other and unspecified intracranial bleeding
                '4321', --subdural
                '4329', --unspecified intracranial bleeding
                --upper gastrointestinal
                '5310', --acute gastric ulcer with bleeding without obstruction
                '5312', --acute gastric ulcer with bleeding and perforation without obstruction
                '5314', --chronic or unspecified gastric ulcer with bleeding without obstruction
                '5316', --chronic or unspecified gastric ulcer with bleeding and perforation without obstruction
                '5320', --acute duodenal ulcer with bleeding without obstruction
                '5322', --acute duodenal ulcer with bleeding and perforation without obstruction
                '5324', --chronic or unspecified duodenal ulcer with bleeding without obstruction
                '5326', --chronic or unspecified duodenal ulcer with bleeding and perforation without obstruction
                '5330', --acute peptic ulcer of unspecified site with bleeding without obstruction
                '5332', --acute peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5334', --chronic or unspecified peptic ulcer of unspecified site with bleeding without obstruction
                '5336', --chronic or unspecified peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5340', --acute gastrojejunal ulcer with bleeding without obstruction
                '5342', --acute gastrojejunal ulcer with bleeding and perforation without obstruction
                '5344', --chronic or unspecified gastrojejunal ulcer with bleeding without obstruction
                '5346', --chronic or unspecified gastrojejunal ulcer with bleeding and perforation without obstruction
                '5780', --hematemesis
                '5781', --blood in stool
                '5789', --bleeding of gastrointestinal tract unspecified
                --Lower GI
                '5693', --bleeding of rectum and anus
                --Other Bleeding
                '2878', --other unspecified hemorrhagic conditions
                '2879', --unspecified hemorrhagic conditions
                '5967', --bleeding into bladder wall
                '7848', --bleeding from throat
                '5997', --hematuria, unspecified
                '6271', --postmenopausal bleeding
                '4590', --bleeding unspecified
                '7191', --hemarthrosis site unspecified
                '7863', --hemoptysis, unspecified
                '72992', --nontraumatic hematoma soft tissue
                '9582' --secondary or recurrent following trauma
            )
        OR icd_code LIKE '900%' --vessel injuries
        OR icd_code LIKE '901%' --vessel injuries
        OR icd_code LIKE '902%' --vessel injuries
        OR icd_code LIKE '903%' --vessel injuries
        OR icd_code LIKE '904%' --vessel injuries

        OR UPPER(LAST_CAREUNIT) LIKE '%TSICU%'
        OR UPPER(FIRST_CAREUNIT) LIKE '%TSICU%' )) )
, demographics AS (
    SELECT
        ie.subject_id,
        ie.hadm_id,
        ie.stay_id -- patient level factors
,
        pat.gender,
        pat.dod -- hospital level factors
,
        adm.admittime,
        adm.dischtime,
        pat.anchor_age AS age,
        adm.admission_type,
        adm.hospital_expire_flag,
        adm.deathtime,
        DENSE_RANK() OVER (
            PARTITION BY adm.subject_id
            ORDER BY
                adm.admittime
        ) AS hospstay_seq,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY adm.subject_id
                ORDER BY
                    adm.admittime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_hosp_stay -- icu level factors
,
        ie.intime,
        ie.outtime,
        DENSE_RANK() OVER (
            PARTITION BY ie.hadm_id
            ORDER BY
                ie.intime
        ) AS icustay_seq -- first ICU stay *for the current hospitalization*
,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id
                ORDER BY
                    ie.intime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_icu_stay
    FROM
        `physionet-data.mimic_icu.icustays` ie
        INNER JOIN `physionet-data.mimic_core.admissions` adm ON ie.hadm_id = adm.hadm_id
        INNER JOIN `physionet-data.mimic_core.patients` pat ON ie.subject_id = pat.subject_id
    ORDER BY
        ie.subject_id,
        adm.admittime,
        ie.intime
)
,
first_adm_offset AS (
    SELECT
        stay_id AS id,
        intime,
        outtime
    FROM
        demographics
    WHERE
        first_icu_stay = TRUE),
pivoted_sofa AS (
    SELECT
        *
    FROM
        `physionet-data.mimic_derived.sofa` ps
        LEFT JOIN first_adm_offset f ON ps.stay_id = f.id
        AND ps.endtime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
pivoted_lab AS (
    SELECT
        *
    FROM
        (
            WITH pivoted_lab AS (
                -- create a table which has fuzzy boundaries on ICU admission (+- 12 hours from documented time)
                -- this is used to assign icustay_id to lab data, which can be collected outside ICU
                -- involves first creating a lag/lead version of intime/outtime
                WITH i AS (
                    SELECT
                        subject_id,
                        stay_id,
                        intime,
                        outtime,
                        lag (outtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS outtime_lag,
                        lead (intime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS intime_lead
                    FROM
                        `physionet-data.mimic_icu.icustays`
                ),
                iid_assign AS (
                    SELECT
                        i.subject_id,
                        i.stay_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN i.outtime_lag IS NOT NULL
                            AND i.outtime_lag > DATETIME_SUB(i.intime, INTERVAL 24 HOUR) THEN DATETIME_SUB(
                                i.intime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime, i.outtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(i.intime, INTERVAL 12 HOUR)
                        END AS data_start,
                        CASE
                            WHEN i.intime_lead IS NOT NULL
                            AND i.intime_lead < DATETIME_ADD(i.outtime, INTERVAL 24 HOUR) THEN DATETIME_ADD(
                                i.outtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime_lead, i.outtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(i.outtime, INTERVAL 12 HOUR)
                        END AS data_end
                    FROM
                        i
                ) -- also create fuzzy boundaries on hospitalization
,
                h AS (
                    SELECT
                        subject_id,hadm_id,admittime,dischtime,
                        lag (dischtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS dischtime_lag,
                        lead (admittime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS admittime_lead
                    FROM
                        `physionet-data.mimic_core.admissions`
                ),
                adm AS (
                    SELECT
                        h.subject_id,
                        h.hadm_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN h.dischtime_lag IS NOT NULL
                            AND h.dischtime_lag > DATETIME_SUB(h.admittime, INTERVAL '24' HOUR) THEN DATETIME_SUB(
                                h.admittime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime, h.dischtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(h.admittime, INTERVAL '12' HOUR)
                        END AS data_start,
                        CASE
                            WHEN h.admittime_lead IS NOT NULL
                            AND h.admittime_lead < DATETIME_ADD(h.dischtime, INTERVAL '24' HOUR) THEN DATETIME_ADD(
                                h.dischtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime_lead, h.dischtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(h.dischtime, INTERVAL '12' HOUR)
                        END AS data_end
                    FROM
                        h
                ),
                le AS (
                    -- begin query that extracts the data
                    SELECT
                        subject_id,
                        charttime -- here we assign labels to ITEMIDs
                        -- this also fuses together multiple ITEMIDs containing the same data
,
                        CASE
                            WHEN itemid = 50868 THEN 'ANION GAP'
                            WHEN itemid = 50862 THEN 'ALBUMIN'
                            WHEN itemid = 51144 THEN 'BANDS'
                            WHEN itemid = 50882 THEN 'BICARBONATE'
                            WHEN itemid = 50885 THEN 'BILIRUBIN'
                            WHEN itemid = 50912 THEN 'CREATININE' -- exclude blood gas
                            -- WHEN itemid = 50806 THEN 'CHLORIDE'
                            WHEN itemid = 50902 THEN 'CHLORIDE' -- exclude blood gas
                            -- WHEN itemid = 50809 THEN 'GLUCOSE'
                            WHEN itemid = 50931 THEN 'GLUCOSE' -- exclude blood gas
                            --WHEN itemid = 50810 THEN 'HEMATOCRIT'
                            WHEN itemid = 51221 THEN 'HEMATOCRIT' -- exclude blood gas
                            --WHEN itemid = 50811 THEN 'HEMOGLOBIN'
                            WHEN itemid = 51222 THEN 'HEMOGLOBIN'
                            WHEN itemid = 50813 THEN 'LACTATE'
                            WHEN itemid = 51265 THEN 'PLATELET' -- exclude blood gas
                            -- WHEN itemid = 50822 THEN 'POTASSIUM'
                            WHEN itemid = 50971 THEN 'POTASSIUM'
                            WHEN itemid = 51275 THEN 'PTT'
                            WHEN itemid = 51237 THEN 'INR'
                            WHEN itemid = 51274 THEN 'PT' -- exclude blood gas
                            -- WHEN itemid = 50824 THEN 'SODIUM'
                            WHEN itemid = 50983 THEN 'SODIUM'
                            WHEN itemid = 51006 THEN 'BUN'
                            WHEN itemid = 51300 THEN 'WBC'
                            WHEN itemid = 51301 THEN 'WBC'
                            ELSE NULL
                        END AS label,
                        -- add in some sanity checks on the values
                        -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
                        CASE
                            WHEN itemid = 50862
                            AND valuenum > 10 THEN NULL -- g/dL 'ALBUMIN'
                            WHEN itemid = 50868
                            AND valuenum > 10000 THEN NULL -- mEq/L 'ANION GAP'
                            WHEN itemid = 51144
                            AND valuenum < 0 THEN NULL -- immature band forms, %
                            WHEN itemid = 51144
                            AND valuenum > 100 THEN NULL -- immature band forms, %
                            WHEN itemid = 50882
                            AND valuenum > 10000 THEN NULL -- mEq/L 'BICARBONATE'
                            WHEN itemid = 50885
                            AND valuenum > 150 THEN NULL -- mg/dL 'BILIRUBIN'
                            WHEN itemid = 50806
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50902
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50912
                            AND valuenum > 150 THEN NULL -- mg/dL 'CREATININE'
                            WHEN itemid = 50809
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50931
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50810
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 51221
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 50811
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 51222
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 50813
                            AND valuenum > 50 THEN NULL -- mmol/L 'LACTATE'
                            WHEN itemid = 51265
                            AND valuenum > 10000 THEN NULL -- K/uL 'PLATELET'
                            WHEN itemid = 50822
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 50971
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 51275
                            AND valuenum > 150 THEN NULL -- sec 'PTT'
                            WHEN itemid = 51237
                            AND valuenum > 50 THEN NULL -- 'INR'
                            WHEN itemid = 51274
                            AND valuenum > 150 THEN NULL -- sec 'PT'
                            WHEN itemid = 50824
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 50983
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 51006
                            AND valuenum > 300 THEN NULL -- 'BUN'
                            WHEN itemid = 51300
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            WHEN itemid = 51301
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            ELSE valuenum
                        END AS valuenum
                    FROM
                        `physionet-data.mimic_hosp.labevents`
                    WHERE
                        ITEMID IN (
                            -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
                            50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
                            50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
                            51144, -- BANDS - hematology
                            50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
                            50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
                            50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
                            50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
                            -- 50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
                            50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
                            -- 50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
                            51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
                            -- 50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
                            51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
                            -- 50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
                            50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
                            51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
                            50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
                            -- 50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
                            51275, -- PTT | HEMATOLOGY | BLOOD | 474937
                            51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
                            51274, -- PT | HEMATOLOGY | BLOOD | 469090
                            50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
                            -- 50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
                            51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
                            51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
                            51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
                        )
                        AND valuenum IS NOT NULL
                        AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
                ),
                le_avg AS (
                    SELECT
                        le.subject_id,
                        le.charttime,
                        avg(
                            CASE
                                WHEN label = 'ANION GAP' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ANIONGAP,
                        avg(
                            CASE
                                WHEN label = 'ALBUMIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ALBUMIN,
                        avg(
                            CASE
                                WHEN label = 'BANDS' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BANDS,
                        avg(
                            CASE
                                WHEN label = 'BICARBONATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BICARBONATE,
                        avg(
                            CASE
                                WHEN label = 'BILIRUBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BILIRUBIN,
                        avg(
                            CASE
                                WHEN label = 'CREATININE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CREATININE,
                        avg(
                            CASE
                                WHEN label = 'CHLORIDE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CHLORIDE,
                        avg(
                            CASE
                                WHEN label = 'GLUCOSE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS GLUCOSE,
                        avg(
                            CASE
                                WHEN label = 'HEMATOCRIT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMATOCRIT,
                        avg(
                            CASE
                                WHEN label = 'HEMOGLOBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMOGLOBIN,
                        avg(
                            CASE
                                WHEN label = 'LACTATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS LACTATE,
                        avg(
                            CASE
                                WHEN label = 'PLATELET' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PLATELET,
                        avg(
                            CASE
                                WHEN label = 'POTASSIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS POTASSIUM,
                        avg(
                            CASE
                                WHEN label = 'PTT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PTT,
                        avg(
                            CASE
                                WHEN label = 'INR' THEN valuenum
                                ELSE NULL
                            END
                        ) AS INR,
                        avg(
                            CASE
                                WHEN label = 'PT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PT,
                        avg(
                            CASE
                                WHEN label = 'SODIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS SODIUM,
                        avg(
                            CASE
                                WHEN label = 'BUN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BUN,
                        avg(
                            CASE
                                WHEN label = 'WBC' THEN valuenum
                                ELSE NULL
                            END
                        ) AS WBC
                    FROM
                        le
                    GROUP BY
                        le.subject_id,
                        le.charttime
                )
                SELECT
                    iid.stay_id,
                    adm.hadm_id,
                    le_avg.*
                FROM
                    le_avg
                    LEFT JOIN adm ON le_avg.subject_id = adm.subject_id
                    AND le_avg.charttime >= adm.data_start
                    AND le_avg.charttime < adm.data_end
                    LEFT JOIN iid_assign iid ON le_avg.subject_id = iid.subject_id
                    AND le_avg.charttime >= iid.data_start
                    AND le_avg.charttime < iid.data_end
                ORDER BY
                    le_avg.subject_id,
                    le_avg.charttime
            )
            SELECT
                *
            FROM
                pivoted_lab
        ) AS pl
        LEFT JOIN first_adm_offset f ON pl.stay_id = f.id
        AND pl.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
)
,
mechvent AS (
    SELECT
        stay_id,
        charttime -- case statement determining whether it is an instance of mech vent
,
        MAX(
            CASE
                WHEN itemid IS NULL
                OR value IS NULL THEN 0 -- can't have null values
                WHEN itemid = 720
                AND value != 'Other/Remarks' THEN 1 -- VentTypeRecorded
                WHEN itemid = 223848
                AND value != 'Other' THEN 1
                WHEN itemid = 223849 THEN 1 -- ventilator mode
                WHEN itemid = 467
                AND value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
                WHEN itemid IN(
                    445,
                    448,
                    449,
                    450,
                    1340,
                    1486,
                    1600,
                    224687 -- minute volume
,
                    639,
                    654,
                    681,
                    682,
                    683,
                    684,
                    224685,
                    224684,
                    224686 -- tidal volume
,
                    218,
                    436,
                    535,
                    444,
                    459,
                    224697,
                    224695,
                    224696,
                    224746,
                    224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
,
                    221,
                    1,
                    1211,
                    1655,
                    2000,
                    226873,
                    224738,
                    224419,
                    224750,
                    227187 -- Insp pressure
,
                    543 -- PlateauPressure
,
                    5865,
                    5866,
                    224707,
                    224709,
                    224705,
                    224706 -- APRV pressure
,
                    60,
                    437,
                    505,
                    506,
                    686,
                    220339,
                    224700 -- PEEP
,
                    3459 -- high pressure relief
,
                    501,
                    502,
                    503,
                    224702 -- PCV
,
                    223,
                    667,
                    668,
                    669,
                    670,
                    671,
                    672 -- TCPCV
,
                    224701 -- PSVlevel
                ) THEN 1
                ELSE 0
            END
        ) AS ventmarker
    FROM
        `physionet-data.mimic_icu.chartevents` ce
        LEFT JOIN first_adm_offset f ON ce.stay_id = f.id
        AND ce.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    WHERE
        ce.value IS NOT NULL -- exclude rows marked as error
        --AND ce.error != 1 -- the below are settings used to indicate ventilation
        AND itemid IN(
            720,
            223849 -- vent mode
,
            223848 -- vent type
,
            445,
            448,
            449,
            450,
            1340,
            1486,
            1600,
            224687 -- minute volume
,
            639,
            654,
            681,
            682,
            683,
            684,
            224685,
            224684,
            224686 -- tidal volume
,
            218,
            436,
            535,
            444,
            224697,
            224695,
            224696,
            224746,
            224747 -- High/Low/Peak/Mean ("RespPressure")
,
            221,
            1,
            1211,
            1655,
            2000,
            226873,
            224738,
            224419,
            224750,
            227187 -- Insp pressure
,
            543 -- PlateauPressure
,
            5865,
            5866,
            224707,
            224709,
            224705,
            224706 -- APRV pressure
,
            60,
            437,
            505,
            506,
            686,
            220339,
            224700 -- PEEP
,
            3459 -- high pressure relief
,
            501,
            502,
            503,
            224702 -- PCV
,
            223,
            667,
            668,
            669,
            670,
            671,
            672 -- TCPCV
,
            224701 -- PSVlevel
            -- the below are settings used to indicate extubation
,
            640 -- extubated
            -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
,
            468 -- O2 Delivery Device#2
,
            469 -- O2 Delivery Mode
,
            470 -- O2 Flow (lpm)
,
            471 -- O2 Flow (lpm) #2
,
            227287 -- O2 Flow (additional cannula)
,
            226732 -- O2 Delivery Device(s)
,
            223834 -- O2 Flow
            -- used in both oxygen + vent calculation
,
            467 -- O2 Delivery Device
        )
    GROUP BY
        stay_id,
        charttime
)
,
sepsis AS (
    -- ICD-9 codes for Angus criteria of sepsis
    SELECT
        hadm_id,
        MAX(
            CASE
                WHEN SUBSTR(icd_code, 1, 3) IN (
                    '001','002','003','004','005',
                    '008','009','010','011','012',
                    '013','014','015','016','017',
                    '018','020','021','022','023',
                    '024','025','026','027','030',
                    '031','032','033','034','035',
                    '036','037','038','039','040',
                    '041','090','091','092','093',
                    '094','095','096','097','098',
                    '100','101','102','103','104',
                    '110','111','112','114','115',
                    '116','117','118','320','322',
                    '324','325','420','421','451',
                    '461','462','463','464','465',
                    '481','482','485','486','494',
                    '510','513','540','541','542',
                    '566','567','590','597','601',
                    '614','615','616','681','682',
                    '683','686','730'
                ) THEN 1
                WHEN SUBSTR(icd_code, 1, 4) IN (
                    '5695','5720','5721','5750',
                    '5990','7110','7907','9966',
                    '9985','9993'
                ) THEN 1
                WHEN SUBSTR(icd_code, 1, 5) IN (
                    '49121','56201','56203',
                    '56211','56213','56983'
                ) THEN 1
                ELSE 0
            END
        ) AS sepsisflag
    FROM
        `physionet-data.mimic_hosp.diagnoses_icd`
    GROUP BY
        hadm_id
),
--,
--vasopressor_duration AS (
    --SELECT
        --*
    --FROM
        --`physionet-data.mimiciii_derived.vasopressordurations`
--),
premorbid AS (
    WITH icd AS (
        SELECT
            hadm_id,
            seq_num,
            CAST(icd_code AS STRING) AS icd_code
        FROM
            `physionet-data.mimic_hosp.diagnoses_icd`
    ),
    eliflg AS (
        SELECT
            hadm_id,
            seq_num,
            icd_code -- note that these codes will seem incomplete at first
            -- for example, CHF is missing a lot of codes referenced in the literature (402.11, 402.91, etc)
            -- these codes are captured by hypertension flags instead
            -- later there are some complicated rules which confirm/reject those codes as CHF
,
            CASE
                WHEN SUBSTR(icd_code, 1, 3) IN(
                    '410', --Acute myocardial infarction
                    '411', --Other acute and subacute forms of ischemic heart disease
                    '412', --Old myocardial infarction
                    '413', --Angina pectoris
                    '414'  --Other forms of chronic ischemic heart disease
                ) THEN 1
            END AS IHD,
            CASE
                WHEN icd_code = '39891' THEN 1
                WHEN icd_code BETWEEN '4280 '
                AND '4289 ' THEN 1
            END AS CHF
            /* Congestive heart failure */
            -- cardiac arrhythmias is removed in up to date versions
,
            CASE
                WHEN icd_code BETWEEN '4400 '
                AND '4409 ' THEN 1
                WHEN icd_code BETWEEN '44100'
                AND '4419 ' THEN 1
                WHEN icd_code BETWEEN '4420 '
                AND '4429 ' THEN 1
                WHEN icd_code BETWEEN '4431 '
                AND '4439 ' THEN 1
                WHEN icd_code BETWEEN '44421'
                AND '44422' THEN 1
                WHEN icd_code = '4471 ' THEN 1
                WHEN icd_code = '449  ' THEN 1
                WHEN icd_code = '5571 ' THEN 1
                WHEN icd_code = '5579 ' THEN 1
                WHEN icd_code = 'V434 ' THEN 1
            END AS PERIVASC
            /* Peripheral vascular disorder */
,
            CASE
                WHEN icd_code = '40200' THEN 1
                WHEN icd_code = '40210' THEN 1
                WHEN icd_code = '40290' THEN 1
                WHEN icd_code = '40509' THEN 1
                WHEN icd_code = '40519' THEN 1
                WHEN icd_code = '40599' THEN 1
            END AS HTNWOCHF
            /* Hypertensive heart disease without heart failure */
,
            CASE
                WHEN icd_code = '40201' THEN 1
                WHEN icd_code = '40211' THEN 1
                WHEN icd_code = '40291' THEN 1
            END AS HTNWCHF
            /* Hypertensive heart disease with heart failure */
,
            CASE
                WHEN icd_code = '40301' THEN 1
                WHEN icd_code = '40311' THEN 1
                WHEN icd_code = '40391' THEN 1
            END AS HRENWRF
            /* Hypertensive renal disease with renal failure */
,
            CASE
                WHEN icd_code = '40401' THEN 1
                WHEN icd_code = '40411' THEN 1
                WHEN icd_code = '40491' THEN 1
            END AS HHRWCHF
            /* Hypertensive heart and renal disease with heart failure */
,
            CASE
                WHEN icd_code = '40402' THEN 1
                WHEN icd_code = '40412' THEN 1
                WHEN icd_code = '40492' THEN 1
            END AS HHRWRF
            /* Hypertensive heart and renal disease with renal failure */
,
            CASE
                WHEN icd_code = '40403' THEN 1
                WHEN icd_code = '40413' THEN 1
                WHEN icd_code = '40493' THEN 1
            END AS HHRWHRF
            /* Hypertensive heart and renal disease with heart and renal failure */
,
            CASE
                WHEN icd_code = '585  ' THEN 1 -- discontinued code
                WHEN icd_code = '5853 ' THEN 1
                WHEN icd_code = '5854 ' THEN 1
                WHEN icd_code = '5855 ' THEN 1
                WHEN icd_code = '5856 ' THEN 1
                WHEN icd_code = '5859 ' THEN 1
                WHEN icd_code = '586  ' THEN 1
                WHEN icd_code = 'V420 ' THEN 1
                WHEN icd_code = 'V451 ' THEN 1
                WHEN icd_code BETWEEN 'V560 '
                AND 'V5632' THEN 1
                WHEN icd_code = 'V568 ' THEN 1
                WHEN icd_code BETWEEN 'V4511'
                AND 'V4512' THEN 1
            END AS RENLFAIL
            /* Renal failure */
        FROM
            icd
    ),
    eligrp AS (
        SELECT
            hadm_id,
            MAX(ihd) AS ihd,
            MAX(chf) AS chf,
            MAX(perivasc) AS perivasc,
            MAX(htnwchf) AS htnwchf,
            MAX(hrenwrf) AS hrenwrf,
            MAX(hhrwchf) AS hhrwchf,
            MAX(hhrwrf) AS hhrwrf,
            MAX(hhrwhrf) AS hhrwhrf,
            MAX(renlfail) AS renlfail
        FROM
            eliflg
        GROUP BY
            hadm_id
    )
    SELECT
        adm.subject_id,
        adm.hadm_id,
        CASE
            WHEN ihd = 1 THEN 1
            ELSE 0
        END AS ISCHEMIC_HEART_DISEASE,
        CASE
            WHEN chf = 1 THEN 1
            WHEN htnwchf = 1 THEN 1
            WHEN hhrwchf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS CONGESTIVE_HEART_FAILURE,
        CASE
            WHEN perivasc = 1 THEN 1
            ELSE 0
        END AS PERIPHERAL_VASCULAR,
        CASE
            WHEN renlfail = 1 THEN 1
            WHEN hrenwrf = 1 THEN 1
            WHEN hhrwrf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS RENAL_FAILURE
    FROM
        `physionet-data.mimic_core.admissions` adm
        LEFT JOIN eligrp eli ON adm.hadm_id = eli.hadm_id
    ORDER BY
        adm.hadm_id
),
surgflag AS (
    SELECT
        adm.hadm_id,
        CASE
            WHEN LOWER(curr_service) LIKE '%surg%' THEN 1
            ELSE 0
        END AS surgical,
        ROW_NUMBER() OVER(
            PARTITION BY adm.HADM_ID
            ORDER BY
                TRANSFERTIME
        ) AS serviceOrder
    FROM
        `physionet-data.mimic_core.admissions` adm
        LEFT JOIN `physionet-data.mimic_hosp.services` se ON adm.hadm_id = se.hadm_id
)
,
tx_raw AS (
    -- BLOOD TRANSFUSED in mL
    -- carevue not in MIMIC 4
    -- include all MIMIC 4 add carvue from MIMIC 3
    WITH inputevents AS (
        SELECT
            ipe.stay_id,
            ipe.starttime AS charttime,
            ROUND(ipe.amount) AS amount,
        FROM
            `physionet-data.mimic_icu.inputevents` ipe
        WHERE
            ipe.itemid IN(
                225168, -- Packed Red Blood Cells
                220996, -- Packed Red Cells
                221013,  -- Whole blood
                226368, -- OR Packed RBC Intake
                227070 -- PACU Packed RBC Intake
            )
            AND amount > 100
            --AND ipe.statusdescription != 'Rewritten'
    )
    SELECT
        ipe.stay_id,
        charttime,
        SUM(amount) AS rbc_volume,
    FROM
        inputevents ipe
    GROUP BY
        ipe.stay_id,
        ipe.charttime
),
tx AS (
    SELECT
        stay_id,
        TIMESTAMP_SECONDS(60 * DIV(UNIX_SECONDS(TIMESTAMP(charttime)), 60)) AS episode,
        --some samples have duplicate readings every 15 minutes - we take an hourly window and average the volume across this window
        --doing so reduces errors significantly
        MIN(charttime) AS charttime,
        --get the first timestamp of each window
        AVG(rbc_volume) AS rbc_volume
    FROM tx_raw
    GROUP BY stay_id, episode
    ORDER BY stay_id, episode
),
tx_counts AS (
    SELECT
        stay_id,
        COUNT(charttime) AS tx_episodes,
        SUM(rbc_volume) AS rbc_volume,
        CEIL(SUM(rbc_volume) / 350) AS rbc_units
    FROM tx
    WHERE
        stay_id IS NOT NULL
    GROUP BY
        stay_id
),
pivoted_hemoglobin AS (
    SELECT
        stay_id,
        charttime,
        hemoglobin
    FROM
        pivoted_lab
    WHERE
        stay_id IS NOT NULL
        AND charttime IS NOT NULL
        AND hemoglobin IS NOT NULL
    ORDER BY
        stay_id
)
, tx_hgb_paired AS (
        SELECT
            w.stay_id,
            w.last_hb_in_window,
            MIN(p.hemoglobin) AS hemoglobin,
            w.tx_offset,
            w.rbc_volume,
            DATETIME_DIFF(w.tx_offset, w.last_hb_in_window, SECOND) AS delta_tx_hgb_seconds,
        FROM
            (
                SELECT
                    p.stay_id,
                    t.charttime AS tx_offset,
                    t.rbc_volume,
                    MAX(p.charttime) AS last_hb_in_window,
                    p.hemoglobin
                FROM tx t --`arctic-welder-307905.blood_transfusion.tx` t
                    INNER JOIN pivoted_hemoglobin p ON p.stay_id = t.stay_id
                    AND p.charttime BETWEEN DATETIME_SUB(t.charttime, INTERVAL 6 HOUR)
                    AND t.charttime --we search the 6 hours prior to transfusion for minimum hemoglobin levels associated with that transfusion
                GROUP BY
                    p.stay_id,
                    t.charttime,
                    t.rbc_volume,
                    p.hemoglobin
                ORDER BY
                    stay_id,
                    last_hb_in_window
            ) w
            INNER JOIN pivoted_hemoglobin p ON p.stay_id = w.stay_id
            AND p.charttime = w.last_hb_in_window
        GROUP BY
            w.stay_id,
            w.last_hb_in_window,
            w.tx_offset,
            w.rbc_volume
        ORDER BY
            stay_id,
             last_hb_in_window,
             tx_offset
)
, tx_hgb_filtered AS (
        SELECT
            *,
            ROW_NUMBER() OVER(x) AS event_number
        FROM tx_hgb_paired
        WINDOW x AS(
                PARTITION BY stay_id
                ORDER BY tx_offset
            )
        ORDER BY stay_id

)
,
tx_mimic_control AS (SELECT stay_id, hadm_id, subject_id, charttime, HEMOGLOBIN
FROM pivoted_lab
WHERE stay_id NOT IN ((SELECT DISTINCT(stay_id) FROM tx_hgb_filtered))),

tx_mimic_control_hb_not_null AS (SELECT * FROM tx_mimic_control
WHERE HEMOGLOBIN IS NOT NULL
ORDER BY stay_id, HEMOGLOBIN),

tx_mimic_control_hb_min_all_columns AS (SELECT *,
ROW_NUMBER() OVER(PARTITION BY stay_id ORDER BY HEMOGLOBIN, charttime ASC) AS RowRank
FROM tx_mimic_control_hb_not_null),

 tx_mimic_control_hb_min AS (SELECT stay_id, MIN(HEMOGLOBIN) as min_hemoglobin
  FROM tx_mimic_control_hb_not_null
  GROUP BY stay_id)

SELECT stay_id, subject_id,charttime, HEMOGLOBIN FROM tx_mimic_control_hb_min_all_columns
Where RowRank=1
ORDER BY stay_id

""").to_dataframe()


In [ ]:
#Hemoglobins level for control group - mimic4
control_mimic4.head()

,stay_id,subject_id,charttime,HEMOGLOBIN
0,30000153,12466550,2174-10-01 05:35:00,8.1
1,30000213,13180007,2162-06-22 04:00:00,7.4
2,30000484,18421337,2136-01-15 04:28:00,8.1
3,30000646,12207593,2194-04-29 02:33:00,12.9
4,30001148,12980335,2156-08-30 13:50:00,8.5


# MIMIC 3 control

In [ ]:
control_mimic3 = client.query("""
WITH
all_criteria AS (SELECT DISTINCT(icustay_id)
FROM `physionet-data.mimiciii_derived.pivoted_sofa`
WHERE icustay_id IN(SELECT DISTINCT(icustay_id)
                    FROM `physionet-data.mimiciii_derived.icustay_detail`
                    WHERE icustay_seq = 1
                    AND first_icu_stay= TRUE
                    AND (admission_age BETWEEN 18 AND 89)
                    )
    AND icustay_id NOT IN(SELECT DISTINCT(i.ICUSTAY_ID)
        FROM `physionet-data.mimiciii_clinical.diagnoses_icd` d
        INNER JOIN `physionet-data.mimiciii_clinical.icustays` i
            ON d.hadm_id=i.hadm_id
        INNER JOIN `physionet-data.mimiciii_derived.icustay_detail` a
            ON d.HADM_ID=a.hadm_id
        WHERE icd9_code IN (
                --ref: https://www.bmj.com/content/bmj/suppl/2015/02/03/bmj.h246.DC1/junm020747.ww1_default.pdf
                --intracranial bleeding
                '430', --Subarachnoid
                '431', --intracerebral
                '4320', --other and unspecified intracranial bleeding
                '4321', --subdural
                '4329', --unspecified intracranial bleeding
                --upper gastrointestinal
                '5310', --acute gastric ulcer with bleeding without obstruction
                '5312', --acute gastric ulcer with bleeding and perforation without obstruction
                '5314', --chronic or unspecified gastric ulcer with bleeding without obstruction
                '5316', --chronic or unspecified gastric ulcer with bleeding and perforation without obstruction
                '5320', --acute duodenal ulcer with bleeding without obstruction
                '5322', --acute duodenal ulcer with bleeding and perforation without obstruction
                '5324', --chronic or unspecified duodenal ulcer with bleeding without obstruction
                '5326', --chronic or unspecified duodenal ulcer with bleeding and perforation without obstruction
                '5330', --acute peptic ulcer of unspecified site with bleeding without obstruction
                '5332', --acute peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5334', --chronic or unspecified peptic ulcer of unspecified site with bleeding without obstruction
                '5336', --chronic or unspecified peptic ulcer of unspecified site with bleeding and perforation without obstruction
                '5340', --acute gastrojejunal ulcer with bleeding without obstruction
                '5342', --acute gastrojejunal ulcer with bleeding and perforation without obstruction
                '5344', --chronic or unspecified gastrojejunal ulcer with bleeding without obstruction
                '5346', --chronic or unspecified gastrojejunal ulcer with bleeding and perforation without obstruction
                '5780', --hematemesis
                '5781', --blood in stool
                '5789', --bleeding of gastrointestinal tract unspecified
                --Lower GI
                '5693', --bleeding of rectum and anus
                --Other Bleeding
                '2878', --other unspecified hemorrhagic conditions
                '2879', --unspecified hemorrhagic conditions
                '5967', --bleeding into bladder wall
                '7848', --bleeding from throat
                '5997', --hematuria, unspecified
                '6271', --postmenopausal bleeding
                '4590', --bleeding unspecified
                '7191', --hemarthrosis site unspecified
                '7863', --hemoptysis, unspecified
                '72992', --nontraumatic hematoma soft tissue
                '9582' --secondary or recurrent following trauma
            )
        OR icd9_code LIKE '900%' --vessel injuries
        OR icd9_code LIKE '901%' --vessel injuries
        OR icd9_code LIKE '902%' --vessel injuries
        OR icd9_code LIKE '903%' --vessel injuries
        OR icd9_code LIKE '904%' --vessel injuries

        OR UPPER(LAST_CAREUNIT) LIKE '%TSICU%'
        OR UPPER(FIRST_CAREUNIT) LIKE '%TSICU%' ))
 ,
 demographics AS (
    SELECT
        ie.subject_id,
        ie.hadm_id,
        ie.icustay_id -- patient level factors
,
        pat.gender,
        pat.dod -- hospital level factors
,
        adm.admittime,
        adm.dischtime,
        DATETIME_DIFF(adm.admittime, pat.dob, YEAR) AS age,
        adm.admission_type,
        adm.hospital_expire_flag,
        adm.deathtime,
        DENSE_RANK() OVER (
            PARTITION BY adm.subject_id
            ORDER BY
                adm.admittime
        ) AS hospstay_seq,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY adm.subject_id
                ORDER BY
                    adm.admittime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_hosp_stay -- icu level factors
,
        ie.intime,
        ie.outtime,
        DENSE_RANK() OVER (
            PARTITION BY ie.hadm_id
            ORDER BY
                ie.intime
        ) AS icustay_seq -- first ICU stay *for the current hospitalization*
,
        CASE
            WHEN DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id
                ORDER BY
                    ie.intime
            ) = 1 THEN TRUE
            ELSE False
        END AS first_icu_stay
    FROM
        `physionet-data.mimiciii_clinical.icustays` ie
        INNER JOIN `physionet-data.mimiciii_clinical.admissions` adm ON ie.hadm_id = adm.hadm_id
        INNER JOIN `physionet-data.mimiciii_clinical.patients` pat ON ie.subject_id = pat.subject_id
    ORDER BY
        ie.subject_id,
        adm.admittime,
        ie.intime
),
first_adm_offset AS (
    SELECT
        icustay_id AS id,
        intime,
        outtime
    FROM
        demographics
    WHERE
        first_icu_stay = TRUE
),
pivoted_sofa AS (
    SELECT
        *
    FROM
        `physionet-data.mimiciii_derived.pivoted_sofa` ps
        LEFT JOIN first_adm_offset f ON ps.icustay_id = f.id
        AND ps.endtime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
pivoted_lab AS (
    SELECT
        *
    FROM
        (
            WITH pivoted_lab AS (
                -- create a table which has fuzzy boundaries on ICU admission (+- 12 hours from documented time)
                -- this is used to assign icustay_id to lab data, which can be collected outside ICU
                -- involves first creating a lag/lead version of intime/outtime
                WITH i AS (
                    SELECT
                        subject_id,
                        icustay_id,
                        intime,
                        outtime,
                        lag (outtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS outtime_lag,
                        lead (intime) over (
                            PARTITION by subject_id
                            ORDER BY
                                intime
                        ) AS intime_lead
                    FROM
                        `physionet-data.mimiciii_clinical.icustays`
                ),
                iid_assign AS (
                    SELECT
                        i.subject_id,
                        i.icustay_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN i.outtime_lag IS NOT NULL
                            AND i.outtime_lag > DATETIME_SUB(i.intime, INTERVAL 24 HOUR) THEN DATETIME_SUB(
                                i.intime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime, i.outtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(i.intime, INTERVAL 12 HOUR)
                        END AS data_start,
                        CASE
                            WHEN i.intime_lead IS NOT NULL
                            AND i.intime_lead < DATETIME_ADD(i.outtime, INTERVAL 24 HOUR) THEN DATETIME_ADD(
                                i.outtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(i.intime_lead, i.outtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(i.outtime, INTERVAL 12 HOUR)
                        END AS data_end
                    FROM
                        i
                ) -- also create fuzzy boundaries on hospitalization
,
                h AS (
                    SELECT
                        subject_id,hadm_id,admittime,dischtime,
                        lag (dischtime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS dischtime_lag,
                        lead (admittime) over (
                            PARTITION by subject_id
                            ORDER BY
                                admittime
                        ) AS admittime_lead
                    FROM
                        `physionet-data.mimiciii_clinical.admissions`
                ),
                adm AS (
                    SELECT
                        h.subject_id,
                        h.hadm_id -- this rule is:
                        --  if there are two hospitalizations within 24 hours, set the start/stop
                        --  time as half way between the two admissions
,
                        CASE
                            WHEN h.dischtime_lag IS NOT NULL
                            AND h.dischtime_lag > DATETIME_SUB(h.admittime, INTERVAL '24' HOUR) THEN DATETIME_SUB(
                                h.admittime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime, h.dischtime_lag, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_SUB(h.admittime, INTERVAL '12' HOUR)
                        END AS data_start,
                        CASE
                            WHEN h.admittime_lead IS NOT NULL
                            AND h.admittime_lead < DATETIME_ADD(h.dischtime, INTERVAL '24' HOUR) THEN DATETIME_ADD(
                                h.dischtime,
                                INTERVAL CAST(
                                    DATETIME_DIFF(h.admittime_lead, h.dischtime, HOUR) / 2 AS INT64
                                ) HOUR
                            )
                            ELSE DATETIME_ADD(h.dischtime, INTERVAL '12' HOUR)
                        END AS data_end
                    FROM
                        h
                ),
                le AS (
                    -- begin query that extracts the data
                    SELECT
                        subject_id,
                        charttime -- here we assign labels to ITEMIDs
                        -- this also fuses together multiple ITEMIDs containing the same data
,
                        CASE
                            WHEN itemid = 50868 THEN 'ANION GAP'
                            WHEN itemid = 50862 THEN 'ALBUMIN'
                            WHEN itemid = 51144 THEN 'BANDS'
                            WHEN itemid = 50882 THEN 'BICARBONATE'
                            WHEN itemid = 50885 THEN 'BILIRUBIN'
                            WHEN itemid = 50912 THEN 'CREATININE' -- exclude blood gas
                            -- WHEN itemid = 50806 THEN 'CHLORIDE'
                            WHEN itemid = 50902 THEN 'CHLORIDE' -- exclude blood gas
                            -- WHEN itemid = 50809 THEN 'GLUCOSE'
                            WHEN itemid = 50931 THEN 'GLUCOSE' -- exclude blood gas
                            --WHEN itemid = 50810 THEN 'HEMATOCRIT'
                            WHEN itemid = 51221 THEN 'HEMATOCRIT' -- exclude blood gas
                            --WHEN itemid = 50811 THEN 'HEMOGLOBIN'
                            WHEN itemid = 51222 THEN 'HEMOGLOBIN'
                            WHEN itemid = 50813 THEN 'LACTATE'
                            WHEN itemid = 51265 THEN 'PLATELET' -- exclude blood gas
                            -- WHEN itemid = 50822 THEN 'POTASSIUM'
                            WHEN itemid = 50971 THEN 'POTASSIUM'
                            WHEN itemid = 51275 THEN 'PTT'
                            WHEN itemid = 51237 THEN 'INR'
                            WHEN itemid = 51274 THEN 'PT' -- exclude blood gas
                            -- WHEN itemid = 50824 THEN 'SODIUM'
                            WHEN itemid = 50983 THEN 'SODIUM'
                            WHEN itemid = 51006 THEN 'BUN'
                            WHEN itemid = 51300 THEN 'WBC'
                            WHEN itemid = 51301 THEN 'WBC'
                            ELSE NULL
                        END AS label,
                        -- add in some sanity checks on the values
                        -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
                        CASE
                            WHEN itemid = 50862
                            AND valuenum > 10 THEN NULL -- g/dL 'ALBUMIN'
                            WHEN itemid = 50868
                            AND valuenum > 10000 THEN NULL -- mEq/L 'ANION GAP'
                            WHEN itemid = 51144
                            AND valuenum < 0 THEN NULL -- immature band forms, %
                            WHEN itemid = 51144
                            AND valuenum > 100 THEN NULL -- immature band forms, %
                            WHEN itemid = 50882
                            AND valuenum > 10000 THEN NULL -- mEq/L 'BICARBONATE'
                            WHEN itemid = 50885
                            AND valuenum > 150 THEN NULL -- mg/dL 'BILIRUBIN'
                            WHEN itemid = 50806
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50902
                            AND valuenum > 10000 THEN NULL -- mEq/L 'CHLORIDE'
                            WHEN itemid = 50912
                            AND valuenum > 150 THEN NULL -- mg/dL 'CREATININE'
                            WHEN itemid = 50809
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50931
                            AND valuenum > 10000 THEN NULL -- mg/dL 'GLUCOSE'
                            WHEN itemid = 50810
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 51221
                            AND valuenum > 100 THEN NULL -- % 'HEMATOCRIT'
                            WHEN itemid = 50811
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 51222
                            AND valuenum > 50 THEN NULL -- g/dL 'HEMOGLOBIN'
                            WHEN itemid = 50813
                            AND valuenum > 50 THEN NULL -- mmol/L 'LACTATE'
                            WHEN itemid = 51265
                            AND valuenum > 10000 THEN NULL -- K/uL 'PLATELET'
                            WHEN itemid = 50822
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 50971
                            AND valuenum > 30 THEN NULL -- mEq/L 'POTASSIUM'
                            WHEN itemid = 51275
                            AND valuenum > 150 THEN NULL -- sec 'PTT'
                            WHEN itemid = 51237
                            AND valuenum > 50 THEN NULL -- 'INR'
                            WHEN itemid = 51274
                            AND valuenum > 150 THEN NULL -- sec 'PT'
                            WHEN itemid = 50824
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 50983
                            AND valuenum > 200 THEN NULL -- mEq/L == mmol/L 'SODIUM'
                            WHEN itemid = 51006
                            AND valuenum > 300 THEN NULL -- 'BUN'
                            WHEN itemid = 51300
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            WHEN itemid = 51301
                            AND valuenum > 1000 THEN NULL -- 'WBC'
                            ELSE valuenum
                        END AS valuenum
                    FROM
                        `physionet-data.mimiciii_clinical.labevents`
                    WHERE
                        ITEMID IN (
                            -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
                            50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
                            50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
                            51144, -- BANDS - hematology
                            50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
                            50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
                            50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
                            50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
                            -- 50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
                            50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
                            -- 50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
                            51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
                            -- 50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
                            51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
                            -- 50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
                            50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
                            51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
                            50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
                            -- 50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
                            51275, -- PTT | HEMATOLOGY | BLOOD | 474937
                            51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
                            51274, -- PT | HEMATOLOGY | BLOOD | 469090
                            50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
                            -- 50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
                            51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
                            51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
                            51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
                        )
                        AND valuenum IS NOT NULL
                        AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
                ),
                le_avg AS (
                    SELECT
                        le.subject_id,
                        le.charttime,
                        avg(
                            CASE
                                WHEN label = 'ANION GAP' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ANIONGAP,
                        avg(
                            CASE
                                WHEN label = 'ALBUMIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS ALBUMIN,
                        avg(
                            CASE
                                WHEN label = 'BANDS' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BANDS,
                        avg(
                            CASE
                                WHEN label = 'BICARBONATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BICARBONATE,
                        avg(
                            CASE
                                WHEN label = 'BILIRUBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BILIRUBIN,
                        avg(
                            CASE
                                WHEN label = 'CREATININE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CREATININE,
                        avg(
                            CASE
                                WHEN label = 'CHLORIDE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS CHLORIDE,
                        avg(
                            CASE
                                WHEN label = 'GLUCOSE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS GLUCOSE,
                        avg(
                            CASE
                                WHEN label = 'HEMATOCRIT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMATOCRIT,
                        avg(
                            CASE
                                WHEN label = 'HEMOGLOBIN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS HEMOGLOBIN,
                        avg(
                            CASE
                                WHEN label = 'LACTATE' THEN valuenum
                                ELSE NULL
                            END
                        ) AS LACTATE,
                        avg(
                            CASE
                                WHEN label = 'PLATELET' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PLATELET,
                        avg(
                            CASE
                                WHEN label = 'POTASSIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS POTASSIUM,
                        avg(
                            CASE
                                WHEN label = 'PTT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PTT,
                        avg(
                            CASE
                                WHEN label = 'INR' THEN valuenum
                                ELSE NULL
                            END
                        ) AS INR,
                        avg(
                            CASE
                                WHEN label = 'PT' THEN valuenum
                                ELSE NULL
                            END
                        ) AS PT,
                        avg(
                            CASE
                                WHEN label = 'SODIUM' THEN valuenum
                                ELSE NULL
                            END
                        ) AS SODIUM,
                        avg(
                            CASE
                                WHEN label = 'BUN' THEN valuenum
                                ELSE NULL
                            END
                        ) AS BUN,
                        avg(
                            CASE
                                WHEN label = 'WBC' THEN valuenum
                                ELSE NULL
                            END
                        ) AS WBC
                    FROM
                        le
                    GROUP BY
                        le.subject_id,
                        le.charttime
                )
                SELECT
                    iid.icustay_id,
                    adm.hadm_id,
                    le_avg.*
                FROM
                    le_avg
                    LEFT JOIN adm ON le_avg.subject_id = adm.subject_id
                    AND le_avg.charttime >= adm.data_start
                    AND le_avg.charttime < adm.data_end
                    LEFT JOIN iid_assign iid ON le_avg.subject_id = iid.subject_id
                    AND le_avg.charttime >= iid.data_start
                    AND le_avg.charttime < iid.data_end
                ORDER BY
                    le_avg.subject_id,
                    le_avg.charttime
            )
            SELECT
                *
            FROM
                pivoted_lab
        ) AS pl
        LEFT JOIN first_adm_offset f ON pl.icustay_id = f.id
        AND pl.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
),
mechvent AS (
    SELECT
        icustay_id,
        charttime -- case statement determining whether it is an instance of mech vent
,
        MAX(
            CASE
                WHEN itemid IS NULL
                OR value IS NULL THEN 0 -- can't have null values
                WHEN itemid = 720
                AND value != 'Other/Remarks' THEN 1 -- VentTypeRecorded
                WHEN itemid = 223848
                AND value != 'Other' THEN 1
                WHEN itemid = 223849 THEN 1 -- ventilator mode
                WHEN itemid = 467
                AND value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
                WHEN itemid IN(
                    445,
                    448,
                    449,
                    450,
                    1340,
                    1486,
                    1600,
                    224687 -- minute volume
,
                    639,
                    654,
                    681,
                    682,
                    683,
                    684,
                    224685,
                    224684,
                    224686 -- tidal volume
,
                    218,
                    436,
                    535,
                    444,
                    459,
                    224697,
                    224695,
                    224696,
                    224746,
                    224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
,
                    221,
                    1,
                    1211,
                    1655,
                    2000,
                    226873,
                    224738,
                    224419,
                    224750,
                    227187 -- Insp pressure
,
                    543 -- PlateauPressure
,
                    5865,
                    5866,
                    224707,
                    224709,
                    224705,
                    224706 -- APRV pressure
,
                    60,
                    437,
                    505,
                    506,
                    686,
                    220339,
                    224700 -- PEEP
,
                    3459 -- high pressure relief
,
                    501,
                    502,
                    503,
                    224702 -- PCV
,
                    223,
                    667,
                    668,
                    669,
                    670,
                    671,
                    672 -- TCPCV
,
                    224701 -- PSVlevel
                ) THEN 1
                ELSE 0
            END
        ) AS ventmarker
    FROM
        `physionet-data.mimiciii_clinical.chartevents` ce
        LEFT JOIN first_adm_offset f ON ce.icustay_id = f.id
        AND ce.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    WHERE
        ce.value IS NOT NULL -- exclude rows marked as error
        AND ce.error != 1 -- the below are settings used to indicate ventilation
        AND itemid IN(
            720,
            223849 -- vent mode
,
            223848 -- vent type
,
            445,
            448,
            449,
            450,
            1340,
            1486,
            1600,
            224687 -- minute volume
,
            639,
            654,
            681,
            682,
            683,
            684,
            224685,
            224684,
            224686 -- tidal volume
,
            218,
            436,
            535,
            444,
            224697,
            224695,
            224696,
            224746,
            224747 -- High/Low/Peak/Mean ("RespPressure")
,
            221,
            1,
            1211,
            1655,
            2000,
            226873,
            224738,
            224419,
            224750,
            227187 -- Insp pressure
,
            543 -- PlateauPressure
,
            5865,
            5866,
            224707,
            224709,
            224705,
            224706 -- APRV pressure
,
            60,
            437,
            505,
            506,
            686,
            220339,
            224700 -- PEEP
,
            3459 -- high pressure relief
,
            501,
            502,
            503,
            224702 -- PCV
,
            223,
            667,
            668,
            669,
            670,
            671,
            672 -- TCPCV
,
            224701 -- PSVlevel
            -- the below are settings used to indicate extubation
,
            640 -- extubated
            -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
,
            468 -- O2 Delivery Device#2
,
            469 -- O2 Delivery Mode
,
            470 -- O2 Flow (lpm)
,
            471 -- O2 Flow (lpm) #2
,
            227287 -- O2 Flow (additional cannula)
,
            226732 -- O2 Delivery Device(s)
,
            223834 -- O2 Flow
            -- used in both oxygen + vent calculation
,
            467 -- O2 Delivery Device
        )
    GROUP BY
        icustay_id,
        charttime
),
sepsis AS (
    -- ICD-9 codes for Angus criteria of sepsis
    SELECT
        hadm_id,
        MAX(
            CASE
                WHEN SUBSTR(icd9_code, 1, 3) IN (
                    '001','002','003','004','005',
                    '008','009','010','011','012',
                    '013','014','015','016','017',
                    '018','020','021','022','023',
                    '024','025','026','027','030',
                    '031','032','033','034','035',
                    '036','037','038','039','040',
                    '041','090','091','092','093',
                    '094','095','096','097','098',
                    '100','101','102','103','104',
                    '110','111','112','114','115',
                    '116','117','118','320','322',
                    '324','325','420','421','451',
                    '461','462','463','464','465',
                    '481','482','485','486','494',
                    '510','513','540','541','542',
                    '566','567','590','597','601',
                    '614','615','616','681','682',
                    '683','686','730'
                ) THEN 1
                WHEN SUBSTR(icd9_code, 1, 4) IN (
                    '5695','5720','5721','5750',
                    '5990','7110','7907','9966',
                    '9985','9993'
                ) THEN 1
                WHEN SUBSTR(icd9_code, 1, 5) IN (
                    '49121','56201','56203',
                    '56211','56213','56983'
                ) THEN 1
                ELSE 0
            END
        ) AS sepsisflag
    FROM
        `physionet-data.mimiciii_clinical.diagnoses_icd`
    GROUP BY
        hadm_id
),
vasopressor_duration AS (
    SELECT
        *
    FROM
        `physionet-data.mimiciii_derived.vasopressordurations`
),
premorbid AS (
    WITH icd AS (
        SELECT
            hadm_id,
            seq_num,
            CAST(icd9_code AS STRING) AS icd9_code
        FROM
            `physionet-data.mimiciii_clinical.diagnoses_icd`
    ),
    eliflg AS (
        SELECT
            hadm_id,
            seq_num,
            icd9_code -- note that these codes will seem incomplete at first
            -- for example, CHF is missing a lot of codes referenced in the literature (402.11, 402.91, etc)
            -- these codes are captured by hypertension flags instead
            -- later there are some complicated rules which confirm/reject those codes as CHF
,
            CASE
                WHEN SUBSTR(icd9_code, 1, 3) IN(
                    '410', --Acute myocardial infarction
                    '411', --Other acute and subacute forms of ischemic heart disease
                    '412', --Old myocardial infarction
                    '413', --Angina pectoris
                    '414'  --Other forms of chronic ischemic heart disease
                ) THEN 1
            END AS IHD,
            CASE
                WHEN icd9_code = '39891' THEN 1
                WHEN icd9_code BETWEEN '4280 '
                AND '4289 ' THEN 1
            END AS CHF
            /* Congestive heart failure */
            -- cardiac arrhythmias is removed in up to date versions
,
            CASE
                WHEN icd9_code BETWEEN '4400 '
                AND '4409 ' THEN 1
                WHEN icd9_code BETWEEN '44100'
                AND '4419 ' THEN 1
                WHEN icd9_code BETWEEN '4420 '
                AND '4429 ' THEN 1
                WHEN icd9_code BETWEEN '4431 '
                AND '4439 ' THEN 1
                WHEN icd9_code BETWEEN '44421'
                AND '44422' THEN 1
                WHEN icd9_code = '4471 ' THEN 1
                WHEN icd9_code = '449  ' THEN 1
                WHEN icd9_code = '5571 ' THEN 1
                WHEN icd9_code = '5579 ' THEN 1
                WHEN icd9_code = 'V434 ' THEN 1
            END AS PERIVASC
            /* Peripheral vascular disorder */
,
            CASE
                WHEN icd9_code = '40200' THEN 1
                WHEN icd9_code = '40210' THEN 1
                WHEN icd9_code = '40290' THEN 1
                WHEN icd9_code = '40509' THEN 1
                WHEN icd9_code = '40519' THEN 1
                WHEN icd9_code = '40599' THEN 1
            END AS HTNWOCHF
            /* Hypertensive heart disease without heart failure */
,
            CASE
                WHEN icd9_code = '40201' THEN 1
                WHEN icd9_code = '40211' THEN 1
                WHEN icd9_code = '40291' THEN 1
            END AS HTNWCHF
            /* Hypertensive heart disease with heart failure */
,
            CASE
                WHEN icd9_code = '40301' THEN 1
                WHEN icd9_code = '40311' THEN 1
                WHEN icd9_code = '40391' THEN 1
            END AS HRENWRF
            /* Hypertensive renal disease with renal failure */
,
            CASE
                WHEN icd9_code = '40401' THEN 1
                WHEN icd9_code = '40411' THEN 1
                WHEN icd9_code = '40491' THEN 1
            END AS HHRWCHF
            /* Hypertensive heart and renal disease with heart failure */
,
            CASE
                WHEN icd9_code = '40402' THEN 1
                WHEN icd9_code = '40412' THEN 1
                WHEN icd9_code = '40492' THEN 1
            END AS HHRWRF
            /* Hypertensive heart and renal disease with renal failure */
,
            CASE
                WHEN icd9_code = '40403' THEN 1
                WHEN icd9_code = '40413' THEN 1
                WHEN icd9_code = '40493' THEN 1
            END AS HHRWHRF
            /* Hypertensive heart and renal disease with heart and renal failure */
,
            CASE
                WHEN icd9_code = '585  ' THEN 1 -- discontinued code
                WHEN icd9_code = '5853 ' THEN 1
                WHEN icd9_code = '5854 ' THEN 1
                WHEN icd9_code = '5855 ' THEN 1
                WHEN icd9_code = '5856 ' THEN 1
                WHEN icd9_code = '5859 ' THEN 1
                WHEN icd9_code = '586  ' THEN 1
                WHEN icd9_code = 'V420 ' THEN 1
                WHEN icd9_code = 'V451 ' THEN 1
                WHEN icd9_code BETWEEN 'V560 '
                AND 'V5632' THEN 1
                WHEN icd9_code = 'V568 ' THEN 1
                WHEN icd9_code BETWEEN 'V4511'
                AND 'V4512' THEN 1
            END AS RENLFAIL
            /* Renal failure */
        FROM
            icd
    ),
    eligrp AS (
        SELECT
            hadm_id,
            MAX(ihd) AS ihd,
            MAX(chf) AS chf,
            MAX(perivasc) AS perivasc,
            MAX(htnwchf) AS htnwchf,
            MAX(hrenwrf) AS hrenwrf,
            MAX(hhrwchf) AS hhrwchf,
            MAX(hhrwrf) AS hhrwrf,
            MAX(hhrwhrf) AS hhrwhrf,
            MAX(renlfail) AS renlfail
        FROM
            eliflg
        GROUP BY
            hadm_id
    )
    SELECT
        adm.subject_id,
        adm.hadm_id,
        CASE
            WHEN ihd = 1 THEN 1
            ELSE 0
        END AS ISCHEMIC_HEART_DISEASE,
        CASE
            WHEN chf = 1 THEN 1
            WHEN htnwchf = 1 THEN 1
            WHEN hhrwchf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS CONGESTIVE_HEART_FAILURE,
        CASE
            WHEN perivasc = 1 THEN 1
            ELSE 0
        END AS PERIPHERAL_VASCULAR,
        CASE
            WHEN renlfail = 1 THEN 1
            WHEN hrenwrf = 1 THEN 1
            WHEN hhrwrf = 1 THEN 1
            WHEN hhrwhrf = 1 THEN 1
            ELSE 0
        END AS RENAL_FAILURE
    FROM
        `physionet-data.mimiciii_clinical.admissions` adm
        LEFT JOIN eligrp eli ON adm.hadm_id = eli.hadm_id
    ORDER BY
        adm.hadm_id
),
surgflag AS (
    SELECT
        adm.hadm_id,
        CASE
            WHEN LOWER(curr_service) LIKE '%surg%' THEN 1
            ELSE 0
        END AS surgical,
        ROW_NUMBER() OVER(
            PARTITION BY adm.HADM_ID
            ORDER BY
                TRANSFERTIME
        ) AS serviceOrder
    FROM
        `physionet-data.mimiciii_clinical.admissions` adm
        LEFT JOIN `physionet-data.mimiciii_clinical.services` se ON adm.hadm_id = se.hadm_id
),
tx_raw AS (
    -- BLOOD TRANSFUSED in mL
    WITH carevue AS (
        SELECT
            cv.icustay_id,
            cv.charttime,
            -- carevue always has units in millilitres (or null)
            ROUND(cv.amount) AS amount
        FROM
            `physionet-data.mimiciii_clinical.inputevents_cv` cv
        WHERE
            cv.itemid IN(
                30001,  -- Packed RBC's
                30002,  -- Whole Blood
                30004,  -- Washed PRBC's
                30104,  -- OR Packed RBC's
                30179,  -- PRBC's
                42588,  -- VICU PRBC
                42239,  -- CC7 PRBC
                42186,	-- Pre admit PRBC
                42239,	-- CC7 PRBC
                42324,	-- er prbc
                42588,	-- VICU PRBC
                42740,	-- prbc in er
                43010,	-- packed cells pacu
                45020,	-- RBC waste
                46124,	-- er in prbc
                46407,	-- ED PRBC
                46612	  -- E.R. prbc
            )
            AND amount > 100 --do not count any transfusions events <100mls
    )
    SELECT
        cv.icustay_id,
        charttime,
        sum(amount) AS rbc_volume,
    FROM
        carevue cv
        LEFT JOIN first_adm_offset f ON cv.icustay_id = f.id
        AND cv.charttime < DATETIME_ADD(f.outtime, INTERVAL 1 HOUR)
    GROUP BY
        cv.icustay_id,
        cv.charttime
),
tx AS (
    SELECT
        icustay_id,
        TIMESTAMP_SECONDS(60 * DIV(UNIX_SECONDS(TIMESTAMP(charttime)), 60)) AS episode,
        --some samples have duplicate readings every 15 minutes - we take an hourly window and average the volume across this window
        --doing so reduces errors significantly
        MIN(charttime) AS charttime,
        --get the first timestamp of each window
        AVG(rbc_volume) AS rbc_volume
    FROM tx_raw
    GROUP BY icustay_id, episode
    ORDER BY icustay_id, episode
),
tx_counts AS (
    SELECT
        icustay_id,
        COUNT(charttime) AS tx_episodes,
        SUM(rbc_volume) AS rbc_volume,
        CEIL(SUM(rbc_volume) / 350) AS rbc_units
    FROM tx
    WHERE
        icustay_id IS NOT NULL
    GROUP BY
        icustay_id
),
pivoted_hemoglobin AS (
    SELECT
        icustay_id,
        charttime,
        hemoglobin
    FROM
        pivoted_lab
    WHERE
        icustay_id IS NOT NULL
        AND charttime IS NOT NULL
        AND hemoglobin IS NOT NULL
    ORDER BY
        icustay_id
)
, tx_hgb_paired AS (
        SELECT
            w.icustay_id,
            w.last_hb_in_window,
            MIN(p.hemoglobin) AS hemoglobin,
            w.tx_offset,
            w.rbc_volume,
            DATETIME_DIFF(w.tx_offset, w.last_hb_in_window, SECOND) AS delta_tx_hgb_seconds,
        FROM
            (
                SELECT
                    p.icustay_id,
                    t.charttime AS tx_offset,
                    t.rbc_volume,
                    MAX(p.charttime) AS last_hb_in_window,
                    p.hemoglobin
                FROM tx t --`arctic-welder-307905.blood_transfusion.tx` t
                    INNER JOIN pivoted_hemoglobin p ON p.icustay_id = t.icustay_id
                    AND p.charttime BETWEEN DATETIME_SUB(t.charttime, INTERVAL 6 HOUR)
                    AND t.charttime --we search the 6 hours prior to transfusion for minimum hemoglobin levels associated with that transfusion
                GROUP BY
                    p.icustay_id,
                    t.charttime,
                    t.rbc_volume,
                    p.hemoglobin
                ORDER BY
                    icustay_id,
                    last_hb_in_window
            ) w
            INNER JOIN pivoted_hemoglobin p ON p.icustay_id = w.icustay_id
            AND p.charttime = w.last_hb_in_window
        GROUP BY
            w.icustay_id,
            w.last_hb_in_window,
            w.tx_offset,
            w.rbc_volume
        ORDER BY
            icustay_id,
             last_hb_in_window,
             tx_offset
)
, tx_hgb_filtered AS (
        SELECT
            *,
            ROW_NUMBER() OVER(x) AS event_number
        FROM tx_hgb_paired
        WINDOW x AS(
                PARTITION BY icustay_id
                ORDER BY tx_offset
            )
        ORDER BY icustay_id

),

tx_mimic_control AS (SELECT icustay_id, hadm_id, subject_id, charttime, HEMOGLOBIN
FROM pivoted_lab
WHERE icustay_id NOT IN ((SELECT DISTINCT(icustay_id) FROM tx_hgb_filtered))),

tx_mimic_control_hb_not_null AS (SELECT * FROM tx_mimic_control
WHERE HEMOGLOBIN IS NOT NULL
ORDER BY icustay_id, HEMOGLOBIN),

tx_mimic_control_hb_min_all_columns AS (SELECT *,
ROW_NUMBER() OVER(PARTITION BY icustay_id ORDER BY HEMOGLOBIN, charttime ASC) AS RowRank
FROM tx_mimic_control_hb_not_null),

 tx_mimic_control_hb_min AS (SELECT icustay_id, MIN(HEMOGLOBIN) as min_hemoglobin
  FROM tx_mimic_control_hb_not_null
  GROUP BY icustay_id)

SELECT icustay_id, subject_id, charttime, HEMOGLOBIN FROM tx_mimic_control_hb_min_all_columns
Where RowRank=1
ORDER BY icustay_id
""").to_dataframe()

In [ ]:
#Hemoglobins level for control group - mimic3
control_mimic3.head()

,icustay_id,subject_id,charttime,HEMOGLOBIN
0,200001,55973,2181-11-26 05:36:00,7.3
1,200003,27513,2199-08-08 01:41:00,9.6
2,200006,10950,2159-09-04 05:21:00,8.7
3,200007,20707,2109-02-18 15:22:00,12.8
4,200010,11861,2132-08-05 02:22:00,8.6


# MIMIC 4 Control Sepsis

In [ ]:
sepsis_mimic4 = client.query("""
SELECT *
FROM `physionet-data.mimic_derived.sepsis3`
""").to_dataframe()

In [ ]:
control_mimic4_septic=pd.merge(control_mimic4,sepsis_mimic4,how='inner',on='stay_id')
#Septic control mimic 4
control_mimic4_septic.head()

,stay_id,subject_id_x,charttime,HEMOGLOBIN,subject_id_y,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,30000484,18421337,2136-01-15 04:28:00,8.1,18421337,2136-01-14 21:00:00,2136-01-14 18:10:00,2136-01-14 18:10:00,2136-01-14 19:00:00,3,0,0,0,0,3,0,True
1,30000646,12207593,2194-04-29 02:33:00,12.9,12207593,2194-04-29 07:00:00,2194-04-29 01:00:00,2194-04-29 01:00:00,2194-04-29 11:00:00,3,2,0,0,1,0,0,True
2,30002415,17921898,2126-12-17 10:04:00,10.4,17921898,2126-12-17 12:00:00,2126-12-16 15:05:00,2126-12-16 15:05:00,2126-12-17 12:00:00,4,2,2,0,0,0,0,True
3,30002654,15978672,2154-10-19 03:34:00,9.4,15978672,2154-10-18 03:00:00,2154-10-17 11:15:00,2154-10-17 11:15:00,2154-10-18 03:00:00,4,0,0,0,4,0,0,True
4,30002925,10208468,2134-06-06 07:00:00,9.7,10208468,2134-06-05 04:00:00,2134-06-04 23:17:00,2134-06-04 23:17:00,2134-06-05 04:00:00,3,0,0,0,1,0,2,True


#MIMIC 3 Control Sepsis

In [ ]:
control_mimic3.rename(columns = {'icustay_id':'stay_id'}, inplace = True)
control_mimic3_carevue_septic=pd.merge(control_mimic3,sepsis_mimiciii,how='inner',on='stay_id')
#Septic control mimic3
control_mimic3_carevue_septic.head()

,stay_id,subject_id,charttime,HEMOGLOBIN,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,...,vent,sofa,lods,sirs,qsofa,qsofa_sysbp_score,qsofa_gcs_score,qsofa_resprate_score,suspicion_poe,sepsis-3
0,200019,21789,2178-07-10 08:20:00,10.8,112486,1,2178-07-08 09:03:12,2178-07-11 10:28:40,carevue,2178-07-08 21:51:00,...,0,5,7,3,2,0.0,1.0,1.0,True,1
1,200047,26135,2190-01-31 05:07:00,12.2,159741,1,2190-01-30 21:47:36,2190-01-31 15:06:57,carevue,2190-01-30 21:30:00,...,0,5,1,1,0,NaN,NaN,NaN,True,1
2,200067,85350,2126-07-01 02:10:00,12.8,122992,1,2126-06-29 18:45:37,2126-07-02 16:00:28,metavision,2126-06-29 20:20:00,...,0,6,7,3,2,1.0,NaN,1.0,True,1
3,200081,51698,2142-03-04 05:23:00,8.3,190004,1,2142-03-01 16:46:10,2142-03-04 18:29:57,metavision,2142-02-27 16:00:00,...,0,2,2,3,2,1.0,NaN,1.0,True,1
4,200099,52619,2163-06-22 02:10:00,9.8,175374,1,2163-06-21 19:49:09,2163-06-23 20:23:11,metavision,2163-06-21 20:39:00,...,0,7,1,2,2,1.0,NaN,1.0,True,1


# SEPTIC Joining MIMIC 4 (METAVISION) with Mimic 3 (carevue) - FULL SEPTIC COHORT CONTROL

In [ ]:
control_group_septic_full = pd.concat([control_mimic3_carevue_septic,control_mimic4_septic])
#FULL SEPTIC GROUP CONTROL
control_group_septic_full.head()

,stay_id,subject_id,charttime,HEMOGLOBIN,hadm_id,excluded,intime,outtime,dbsource,suspected_infection_time_poe,...,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,200019,21789.0,2178-07-10 08:20:00,10.8,112486.0,1.0,2178-07-08 09:03:12,2178-07-11 10:28:40,carevue,2178-07-08 21:51:00,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200047,26135.0,2190-01-31 05:07:00,12.2,159741.0,1.0,2190-01-30 21:47:36,2190-01-31 15:06:57,carevue,2190-01-30 21:30:00,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200067,85350.0,2126-07-01 02:10:00,12.8,122992.0,1.0,2126-06-29 18:45:37,2126-07-02 16:00:28,metavision,2126-06-29 20:20:00,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200081,51698.0,2142-03-04 05:23:00,8.3,190004.0,1.0,2142-03-01 16:46:10,2142-03-04 18:29:57,metavision,2142-02-27 16:00:00,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200099,52619.0,2163-06-22 02:10:00,9.8,175374.0,1.0,2163-06-21 19:49:09,2163-06-23 20:23:11,metavision,2163-06-21 20:39:00,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Next Steps - from here

- Check all variables in paper
- Looking at the probability of transfusion over time ??
- Probability of transfusion across gender and across race-ethnicity because those are the ones that we will have to consider when we start looking at the question does transfusion help or not
-Use casual inference methodology (TMLE) and multiply robust estimation to estimate the causal effect of transfusion among non bleeding patients with sepsis


#MIMIC 4 ethnicities

In [ ]:
mimic4_ethnicities = client.query("""
SELECT hadm_id, ethnicity
FROM `physionet-data.mimic_core.admissions`
""").to_dataframe()

In [ ]:
mimic4_ethnicities.head()

,hadm_id,ethnicity
0,21292863,AMERICAN INDIAN/ALASKA NATIVE
1,25693393,AMERICAN INDIAN/ALASKA NATIVE
2,25137562,AMERICAN INDIAN/ALASKA NATIVE
3,23443808,AMERICAN INDIAN/ALASKA NATIVE
4,23324841,AMERICAN INDIAN/ALASKA NATIVE


#MIMIC 3 ethnicities

In [ ]:
mimic3_ethnicities = client.query("""
SELECT hadm_id,ethnicity
FROM `physionet-data.mimiciii_clinical.admissions`
""").to_dataframe()

In [ ]:
mimic3_ethnicities.head()

,hadm_id,ethnicity
0,180546,HISPANIC/LATINO - PUERTO RICAN
1,149469,HISPANIC/LATINO - PUERTO RICAN
2,153826,HISPANIC/LATINO - PUERTO RICAN
3,192349,ASIAN - CHINESE
4,191209,HISPANIC/LATINO - PUERTO RICAN


#Merge ethnicities to obtain a full cohort

In [ ]:
ethnicities = pd.concat([mimic4_ethnicities,mimic3_ethnicities])
ethnicities.head()

,hadm_id,ethnicity
0,21292863,AMERICAN INDIAN/ALASKA NATIVE
1,25693393,AMERICAN INDIAN/ALASKA NATIVE
2,25137562,AMERICAN INDIAN/ALASKA NATIVE
3,23443808,AMERICAN INDIAN/ALASKA NATIVE
4,23324841,AMERICAN INDIAN/ALASKA NATIVE


In [ ]:
treatment_group_septic_w_ethnicity=treatment_group_septic.merge(ethnicities, on='hadm_id', how='inner')
control_group_septic_w_ethnicity=control_group_septic_full.merge(ethnicities, on='hadm_id', how='inner')

In [ ]:
len(treatment_group_septic)

8641

#OLD STUFF TO CHECK

# Mimic 4 Subject id of patients with more than one ethnicity

In [ ]:
Subject_id_multiple_ethnicity_mimic4= client.query("""
with patients_more_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimic_core.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>1)

Select * from t3),

distinct_ethnicity_subject_id as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimic_core.admissions`
WHERE subject_id in (Select subject_id from patients_more_ethnicity)
order by subject_id),

subject_id_more_one_ethnicity as (Select distinct subject_id
from distinct_ethnicity_subject_id)

Select * from Subject_id_more_one_ethnicity
""").to_dataframe()


In [ ]:
#List of subjects id that have listed more ethnicities in the db
len(Subject_id_multiple_ethnicity_mimic4)

6333

In [ ]:
Subject_id_ethnicity_mimic4= client.query("""
with patients_more_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimic_core.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>1)

Select * from t3),

distinct_subjectid_ethnicity as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimic_core.admissions`
WHERE subject_id in (Select subject_id from patients_more_ethnicity)
order by subject_id)

Select * from distinct_subjectid_ethnicity

""").to_dataframe()

In [ ]:
Subject_id_ethnicity_mimic4)

,subject_id,ethnicity
0,10002013,WHITE
1,10002013,OTHER
2,10002869,UNKNOWN
3,10002869,WHITE
4,10004457,WHITE
...,...,...
12818,19994592,OTHER
12819,19996675,WHITE
12820,19996675,UNKNOWN
12821,19999287,BLACK/AFRICAN AMERICAN


In [ ]:
import pandas as pd
control_group_septic_full.merge(Subject_id_multiple_ethnicity, on= 'subject_id', how= 'inner' )

NameError: ignored

In [ ]:
import pandas as pd
treatment_group_septic.merge(Subject_id_multiple_ethnicity, on= 'subject_id', how= 'inner' )

NameError: ignored

In [ ]:
#Patients with majority ethnicity tied
majority_ethjnicity_tied_mimic4= client.query("""
with t as (with t1 as (SELECT a.subject_id, a.ethnicity
 FROM `physionet-data.mimic_core.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>2)

Select * from t3),


subjectid_ethnicity as (SELECT subject_id, ethnicity FROM `physionet-data.mimic_core.admissions`
WHERE subject_id in (Select subject_id from t)
order by subject_id)
,
subjectid_ethnicity_frequency as (SELECT subject_id, ethnicity, COUNT(*)  as frequency,
FROM subjectid_ethnicity
GROUP BY subject_id, ethnicity
ORDER BY subject_id, frequency)
,
subjectid_ethnicity_ranked as (SELECT subject_id, ethnicity, frequency,
DENSE_RANK() OVER(PARTITION BY subject_id ORDER BY frequency DESC) as most_common_ethnicity
FROM subjectid_ethnicity_frequency
ORDER BY subject_id, most_common_ethnicity)
,
duplicate_ethnicity_subjects as (SELECT subject_id, COUNT(*) as duplicate
FROM subjectid_ethnicity_ranked
WHERE most_common_ethnicity=1
GROUP BY subject_id HAVING COUNT(*)>1
ORDER BY subject_id)

SELECT *
FROM subjectid_ethnicity_ranked a
INNER JOIN duplicate_ethnicity_subjects b ON a.subject_id=b.subject_id
""").to_dataframe()

# MIMIC3

In [ ]:
Subject_id_more_one_ethnicity_mimic3= client.query("""
with patients_more_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimiciii_clinical.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>1)

Select * from t3),

distinct_ethnicity_subject_id as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimiciii_clinical.admissions`
WHERE subject_id in (Select subject_id from patients_more_ethnicity)
order by subject_id),

subject_id_more_one_ethnicity as (Select distinct subject_id
from distinct_ethnicity_subject_id)

Select * from Subject_id_more_one_ethnicity""").to_dataframe()


In [ ]:
#Subjects with multiple ethnicities
Subject_id_more_one_ethnicity_mimic3

,subject_id
0,76476
1,426
2,1018
3,794
4,2093
...,...
284,12746
285,21423
286,22578
287,73385


In [ ]:
Subject_id_ethnicity_mimic3= client.query("""with patients_more_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimiciii_clinical.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>1)

Select * from t3),

distinct_subjectid_ethnicity as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimiciii_clinical.admissions`
WHERE subject_id in (Select subject_id from patients_more_ethnicity)
order by subject_id)

Select * from distinct_subjectid_ethnicity

""").to_dataframe()

In [ ]:
#Retrieve the ethnicities for the patients with multiple ethnicities
Subject_id_ethnicity_mimic3

,subject_id,ethnicity
0,191,ASIAN
1,191,ASIAN - CHINESE
2,426,UNKNOWN/NOT SPECIFIED
3,426,WHITE
4,518,OTHER
...,...,...
575,97659,WHITE
576,98347,UNKNOWN/NOT SPECIFIED
577,98347,WHITE
578,99088,WHITE - RUSSIAN


In [ ]:
#Split composite ethnicities into its subsets
Subject_id_ethnicity_mimic3[['general ethnicity', 'specific']] = Subject_id_ethnicity_mimic3['ethnicity'].str.split('-', expand=True)

In [ ]:
#Dataframe with main ethnicity group for patient with multiple ethnicities
general_ethnicity= Subject_id_ethnicity_mimic3[['subject_id','general ethnicity']]
general_ethnicity

,subject_id,general ethnicity
0,191,ASIAN
1,191,ASIAN
2,426,UNKNOWN/NOT SPECIFIED
3,426,WHITE
4,518,OTHER
...,...,...
575,97659,WHITE
576,98347,UNKNOWN/NOT SPECIFIED
577,98347,WHITE
578,99088,WHITE


In [ ]:
#Get rid of empty spaces in the string
import pandas as pd
general_ethnicity['general ethnicity']=general_ethnicity['general ethnicity'].str.replace(' ', '')
# convert series to dataframe
general_ethnicity= pd.DataFrame(general_ethnicity)

In [ ]:
general_ethnicity

,subject_id,general ethnicity
0,191,ASIAN
1,191,ASIAN
2,426,UNKNOWN/NOTSPECIFIED
3,426,WHITE
4,518,OTHER
...,...,...
575,97659,WHITE
576,98347,UNKNOWN/NOTSPECIFIED
577,98347,WHITE
578,99088,WHITE


In [ ]:
#general_ethnicity['general ethnicity'].str.strip()

In [ ]:
general_ethnicity=general_ethnicity.drop_duplicates()

In [ ]:
#Patients with multiple ethnicity in mimic 3
general_ethnicity

,subject_id,general ethnicity
0,191,ASIAN
2,426,UNKNOWN/NOTSPECIFIED
3,426,WHITE
4,518,OTHER
5,518,BLACK/AFRICAN
...,...,...
574,97659,UNKNOWN/NOTSPECIFIED
575,97659,WHITE
576,98347,UNKNOWN/NOTSPECIFIED
577,98347,WHITE


In [ ]:
#Patient with majority ethnicity tied in MIMIC-III
majority_ethjnicity_tied_mimic3= client.query("""
--IF WHITE AND NON-WHITE THEN NON-WHITE, IF WHITE AND UNKNOWN THEN WHITE, IF NON-WHITE AND UNKNOWN THEN NON-WHITE
--NO PATIENT WITH 3 DIFFERENT CATEGORIES TIED, JUST TIES BETWEEN MAX 2 CATEGORIES
WITH admissions_ethnicity_categorized AS (WITH ethnicity_dictionary as (SELECT DISTINCT ethnicity,
CASE
  WHEN ethnicity LIKE '%WHITE%' THEN 'WHITE'
  --WHEN ethnicity LIKE '%ASIAN%' THEN 'ASIAN'
  --WHEN ethnicity LIKE '%HISPANIC%' THEN 'HISPANIC'
  --WHEN ethnicity LIKE '%BLACK%' THEN 'BLACK'
  WHEN ethnicity IN ('PATIENT DECLINED TO ANSWER','UNKNOWN/NOT SPECIFIED','UNABLE TO OBTAIN') THEN 'UNKNOWN'
  ELSE 'NON-WHITE'
END
as ethnicity_categorization
FROM `physionet-data.mimiciii_clinical.admissions`)

SELECT adm.*, dic.ethnicity_categorization
FROM `physionet-data.mimiciii_clinical.admissions` as adm
INNER JOIN ethnicity_dictionary dic ON adm.ethnicity=dic.ethnicity)
,
t as (with t1 as (SELECT cat.subject_id, cat.ethnicity_categorization
 FROM admissions_ethnicity_categorized cat),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>2)

Select * from t3),


subjectid_ethnicity as (SELECT subject_id, ethnicity_categorization FROM admissions_ethnicity_categorized
WHERE subject_id in (Select subject_id from t)
order by subject_id)
,
subjectid_ethnicity_frequency as (SELECT subject_id, ethnicity_categorization, COUNT(*)  as frequency,
FROM subjectid_ethnicity
GROUP BY subject_id, ethnicity_categorization
ORDER BY subject_id, frequency)
,
subjectid_ethnicity_ranked as (SELECT subject_id, ethnicity_categorization, frequency,
DENSE_RANK() OVER(PARTITION BY subject_id ORDER BY frequency DESC) as most_common_ethnicity
FROM subjectid_ethnicity_frequency
ORDER BY subject_id, most_common_ethnicity)
,
duplicate_ethnicity_subjects as (SELECT subject_id, COUNT(*) as duplicate
FROM subjectid_ethnicity_ranked
WHERE most_common_ethnicity=1
GROUP BY subject_id HAVING COUNT(*)>1
ORDER BY subject_id)

SELECT *
FROM subjectid_ethnicity_ranked a
INNER JOIN duplicate_ethnicity_subjects b ON a.subject_id=b.subject_id
""").to_dataframe()

In [ ]:
#Solve with the previous rule, to solve ties between the majority ethnicities
import pandas as pd
import numpy as np
length=majority_ethjnicity_tied_mimic3['subject_id'].unique()
ethnicity_ties_solved = pd.DataFrame(columns=['subject_id', 'ethnicity'], index=range(length.size))
k=0
j=0
for i in length:
  ethnicity_list_temp=majority_ethjnicity_tied_mimic3[majority_ethjnicity_tied_mimic3['subject_id']==i]
  ethnicity_ties_solved.iloc[k]['subject_id']=length[k]
  if (ethnicity_list_temp['ethnicity_categorization'][j]=='NON-WHITE')|(ethnicity_list_temp['ethnicity_categorization'][j+1]=='NON-WHITE'):
    ethnicity_ties_solved.iloc[k]['ethnicity']='NON-WHITE'
  else:
    ethnicity_ties_solved.iloc[k]['ethnicity']='WHITE'
  k=k+1
  j=j+2
ethnicity_ties_solved

,subject_id,ethnicity
0,12110,NON-WHITE
1,32447,NON-WHITE
2,29866,NON-WHITE
3,4268,NON-WHITE
4,7029,NON-WHITE
5,21734,WHITE
6,16275,NON-WHITE
7,68127,NON-WHITE


# June 06 2022

In [ ]:
Subject_id_ethnicity_mimic3= client.query("""with patients_more_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimiciii_clinical.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq>1)

Select * from t3),

distinct_subjectid_ethnicity as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimiciii_clinical.admissions`
WHERE subject_id in (Select subject_id from patients_more_ethnicity)
order by subject_id)

Select * from distinct_subjectid_ethnicity

""").to_dataframe()

In [ ]:
Subject_id_one_ethnicity_mimic3= client.query("""with patients_one_ethnicity as (with t1 as (SELECT DISTINCT a.subject_id, a.ethnicity
 FROM `physionet-data.mimiciii_clinical.admissions` a),

t3 as (SELECT subject_id, COUNT(*) AS freq
from t1
group by subject_id
having freq=1)

Select * from t3),

distinct_subjectid_ethnicity as (SELECT distinct subject_id, ethnicity FROM `physionet-data.mimiciii_clinical.admissions`
WHERE subject_id in (Select subject_id from patients_one_ethnicity)
order by subject_id)

Select subject_id,
CASE
  WHEN ethnicity LIKE '%WHITE%' THEN 'WHITE'
  WHEN ethnicity IN ('PATIENT DECLINED TO ANSWER','UNKNOWN/NOT SPECIFIED','UNABLE TO OBTAIN') THEN 'UNKNOWN'
  ELSE 'NON-WHITE'
END
as ethnicity
 from distinct_subjectid_ethnicity
 """).to_dataframe()

In [ ]:
#Uniforming white rows
Subject_id_ethnicity_mimic3.loc[Subject_id_ethnicity_mimic3['ethnicity'].str.contains('WHITE'), 'ethnicity'] = 'WHITE'